In [1]:
import warnings
warnings.filterwarnings('ignore')
from numpy import set_printoptions
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
pd.options.display.max_columns=None
import seaborn as sns 
from pandas import read_csv
import io
import base64
import json
from openpyxl import Workbook
from openpyxl.drawing.image import Image as XLImage

import os
import os.path
from unidecode import unidecode

from urllib.error import HTTPError

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload
from sklearn.model_selection import StratifiedKFold

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression 
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from mlens.ensemble import SuperLearner


from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import normalized_mutual_info_score
from sklearn.metrics import cohen_kappa_score


[MLENS] backend: threading


### CARGUE DE DATOS Y CONEXIÓN A DRIVE

In [2]:
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

creds = None
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/credentials.json', SCOPES) # Reemplazar con la ruta correcta
        creds = flow.run_local_server(port=0)
    with open('C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/token.json', 'w') as token:
        token.write(creds.to_json())
        
# Crear una instancia de la API de Drive
drive_service = build('drive', 'v3', credentials=creds)

# ID de la carpeta de Google Drive
folder_id = '1hQeetmO4XIObUefS_nzePqKqq3VksUEC'

# Ruta de destino para guardar los archivos descargados
save_path = 'C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/UNIVERSIDAD-DISTRITAL-PROYECTO/MÓDULO ANALÍTICA PREDICTIVA/DATOS'  # Reemplazar con la ruta deseada

# Función para descargar archivos de la carpeta de Drive
def download_folder(folder_id, save_path):
    results = drive_service.files().list(
        q=f"'{folder_id}' in parents and trashed=false",
        fields='files(id, name)').execute()
    items = results.get('files', [])
    for item in items:
        file_id = item['id']
        file_name = item['name']
        request = drive_service.files().get_media(fileId=file_id)
        fh = io.FileIO(os.path.join(save_path, file_name), 'wb')
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print(f"Descargando {file_name}: {int(status.progress() * 100)}%")
    print("Descarga completa")

# Descargar archivos de la carpeta
download_folder(folder_id, save_path)

# Listar archivos descargados
files = os.listdir(save_path)
print("Archivos descargados:")
for file in files:
    print(file)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=899973764197-biu188dkvsgi2al0fh29udm7keak0lh0.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A59663%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&state=GXyQBQylOtgePa4lgIt1IizTkFts08&access_type=offline


Descargando catastral2.csv: 100%
Descargando catastral5.csv: 100%
Descargando electronica7.csv: 100%
Descargando electronica5.csv: 100%
Descargando catastral10.csv: 100%
Descargando catastral9.csv: 100%
Descargando catastral8.csv: 100%
Descargando catastral3.csv: 100%
Descargando catastral7.csv: 100%
Descargando catastral6.csv: 100%
Descargando catastral4.csv: 100%
Descargando electronica10.csv: 100%
Descargando electronica9.csv: 100%
Descargando electronica8.csv: 100%
Descargando electronica6.csv: 100%
Descargando electronica4.csv: 100%
Descargando electronica3.csv: 100%
Descargando electronica2.csv: 100%
Descargando electronica1.csv: 100%
Descargando Limpieza de datos 1 UD.ipynb: 100%
Descargando industrial10.csv: 100%
Descargando industrial9.csv: 100%
Descargando industrial8.csv: 100%
Descargando industrial7.csv: 100%
Descargando industrial6.csv: 100%
Descargando industrial5.csv: 100%
Descargando industrial3.csv: 100%
Descargando industrial4.csv: 100%
Descargando industrial2.csv: 10

In [ ]:
"""
variables_por_carrera = {
    'industrial': {
        '1': ['PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES','QUIMICA_ICFES','IDIOMA_ICFES','LOCALIDAD','RENDIMIENTO_UNO', 'PROMEDIO_UNO'],
        '2': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES', 'BIOLOGIA_ICFES', 'IDIOMA_ICFES', 'LOCALIDAD', 'PROMEDIO_UNO', 'CAR_UNO', 'NCC_UNO', 'NAA_UNO', 'NOTA_DIFERENCIAL', 'NOTA_DIBUJO', 'NOTA_QUIMICA', 'NOTA_CFJC', 'NOTA_TEXTOS', 'NOTA_SEMINARIO', 'NOTA_EE_UNO','RENDIMIENTO_DOS', 'PROMEDIO_DOS'],
        '3': ['PROMEDIO_UNO', 'NAA_UNO', 'NOTA_DIFERENCIAL', 'NOTA_DIBUJO', 'NOTA_TEXTOS', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NCC_DOS', 'NCA_DOS', 'NAA_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_PBASICA', 'NOTA_EE_DOS', 'RENDIMIENTO_TRES', 'PROMEDIO_TRES'],
        '4': ['PROMEDIO_UNO', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_ESTADISTICA_UNO', 'NOTA_TERMODINAMICA', 'NOTA_TGS', 'NOTA_EE_TRES','RENDIMIENTO_CUATRO', 'PROMEDIO_CUATRO'],
        '5': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS','PROMEDIO_TRES', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_TERMODINAMICA', 'NOTA_ECUACIONES', 'NOTA_ESTADISTICA_DOS', 'NOTA_FISICA_DOS', 'NOTA_MECANICA', 'NOTA_PROCESOSQ', 'RENDIMIENTO_CINCO', 'RENDIMIENTO_CINCO'],
        '6': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_ADMINISTRACION', 'NOTA_LENGUA_UNO', 'NOTA_EI_UNO', 'NOTA_EI_DOS', 'RENDIMIENTO_SEIS', 'PROMEDIO_SEIS'],
        '7': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_LENGUA_UNO', 'NOTA_EI_DOS', 'PROMEDIO_SEIS', 'NCA_SEIS', 'NOTA_PLINEAL', 'NOTA_DISE O', 'NOTA_EI_TRES','RENDIMIENTO_SIETE', 'PROMEDIO_SIETE'],
        '8': ['PROMEDIO_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_LENGUA_UNO','PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'RENDIMIENTO_OCHO', 'PROMEDIO_OCHO'],
        '9': ['PROMEDIO_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'PROMEDIO_OCHO', 'NCC_OCHO', 'NOTA_LOG_UNO', 'NOTA_GOPERACIONES','NOTA_CALIDAD_DOS', 'NOTA_LENGUA_DOS', 'NOTA_CONTEXTO', 'RENDIMIENTO_NUEVE', 'PROMEDIO_NUEVE'],
        '10': ['PROMEDIO_SEIS', 'PROMEDIO_SIETE', 'PROMEDIO_OCHO', 'NOTA_CALIDAD_DOS', 'PROMEDIO_NUEVE', 'NAA_NUEVE', 'NOTA_GRADO_UNO', 'NOTA_LOG_DOS', 'NOTA_FINANZAS', 'NOTA_HISTORIA', 'RENDIMIENTO_DIEZ', 'PROMEDIO_DIEZ']
    },
    'sistemas': {
        '1': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES', 'QUIMICA_ICFES', 'IDIOMA_ICFES', 'LOCALIDAD', 'RENDIMIENTO_UNO', 'PROMEDIO_UNO'],
        '2': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'IDIOMA_ICFES', 'PROMEDIO_UNO', 'NOTA_DIFERENCIAL', 'NOTA_PROG_BASICA', 'NOTA_CATEDRA_FJC', 'NOTA_TEXTOS', 'NOTA_SEMINARIO', 'NOTA_CATEDRA_DEM', 'NOTA_CATEDRA_CON', 'NOTA_LOGICA', 'RENDIMIENTO_DOS', 'PROMEDIO_DOS'],
        '3': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'IDIOMA_ICFES', 'PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_PROG_BASICA', 'NOTA_TEXTOS', 'NOTA_CATEDRA_DEM', 'NOTA_INTEGRAL', 'NOTA_PROG_ORIENTADA', 'NOTA_ETICA', 'RENDIMIENTO_TRES', 'PROMEDIO_TRES'],
        '4': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'IDIOMA_ICFES', 'PROMEDIO_UNO', 'PROMEDIO_DOS', 'PROMEDIO_TRES', 'NOTA_PROG_BASICA', 'NOTA_TEXTOS', 'NOTA_CATEDRA_DEM', 'NOTA_INTEGRAL', 'NOTA_PROG_ORIENTADA', 'NOTA_ETICA', 'NOTA_FISICA_DOS', 'NOTA_TGS', 'NOTA_PROG_AVANZADA', 'RENDIMIENTO_CUATRO', 'PROMEDIO_CUATRO'],
        '5': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_TERMODINAMICA', 'NOTA_ECUACIONES', 'NOTA_ESTADISTICA_DOS', 'NOTA_FISICA_DOS', 'NOTA_MECANICA', 'NOTA_PROCESOSQ', 'RENDIMIENTO_CINCO', 'PROMEDIO_CINCO'],
        '6': ['PROMEDIO_UNO', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_ECONOMIA_UNO', 'NOTA_FISICA_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_ADMINISTRACION', 'NOTA_LENGUA_UNO', 'NOTA_EI_UNO', 'NOTA_EI_DOS', 'RENDIMIENTO_SEIS', 'PROMEDIO_SEIS'],
        '7': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_LENGUA_UNO', 'NOTA_EI_DOS', 'PROMEDIO_SEIS', 'NCA_SEIS', 'NOTA_PLINEAL', 'NOTA_DISE O', 'NOTA_EI_TRES', 'RENDIMIENTO_SIETE', 'PROMEDIO_SIETE'],
        '8': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_LENGUA_UNO', 'PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'RENDIMIENTO_OCHO', 'PROMEDIO_OCHO'],
        '9': ['PROMEDIO_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'PROMEDIO_OCHO', 'NCC_OCHO', 'NOTA_LOG_UNO', 'NOTA_GOPERACIONES', 'NOTA_CALIDAD_DOS', 'NOTA_LENGUA_DOS', 'NOTA_CONTEXTO', 'RENDIMIENTO_NUEVE', 'PROMEDIO_NUEVE'],
        '10': ['PROMEDIO_SEIS', 'PROMEDIO_SIETE', 'PROMEDIO_OCHO', 'NOTA_CALIDAD_DOS', 'PROMEDIO_NUEVE', 'NAA_NUEVE', 'NOTA_GRADO_UNO', 'NOTA_LOG_DOS', 'NOTA_FINANZAS', 'NOTA_HISTORIA', 'RENDIMIENTO_DIEZ', 'PROMEDIO_DIEZ']
    },
    'catastral': {
        '1': ['variable1_catastral', 'variable2_catastral', 'variable3_catastral'],
        '2': ['variable4_catastral', 'variable5_catastral', 'variable6_catastral']
    }
}
"""

"\nvariables_por_carrera = {\n    'industrial': {\n        '1': ['PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES','QUIMICA_ICFES','IDIOMA_ICFES','LOCALIDAD','RENDIMIENTO_UNO', 'PROMEDIO_UNO'],\n        '2': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES', 'BIOLOGIA_ICFES', 'IDIOMA_ICFES', 'LOCALIDAD', 'PROMEDIO_UNO', 'CAR_UNO', 'NCC_UNO', 'NAA_UNO', 'NOTA_DIFERENCIAL', 'NOTA_DIBUJO', 'NOTA_QUIMICA', 'NOTA_CFJC', 'NOTA_TEXTOS', 'NOTA_SEMINARIO', 'NOTA_EE_UNO','RENDIMIENTO_DOS', 'PROMEDIO_DOS'],\n        '3': ['PROMEDIO_UNO', 'NAA_UNO', 'NOTA_DIFERENCIAL', 'NOTA_DIBUJO', 'NOTA_TEXTOS', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NCC_DOS', 'NCA_DOS', 'NAA_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_PBASICA', 'NOTA_EE_DOS', 'RENDIMIENTO_TRES', 'PROMEDIO_TRES'],\n        '4': ['PROMEDIO_UNO', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_ESTADISTICA_UNO', 'NOTA_TERMODINAMICA', 'NO

In [5]:
variables_por_carrera = { 
    'industrial': {
        '1': ['PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES','QUIMICA_ICFES','IDIOMA_ICFES','LOCALIDAD','RENDIMIENTO_UNO'],
        '2': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES', 'BIOLOGIA_ICFES', 'IDIOMA_ICFES', 'LOCALIDAD', 'PROMEDIO_UNO', 'CAR_UNO', 'NCC_UNO', 'NAA_UNO', 'NOTA_DIFERENCIAL', 'NOTA_DIBUJO', 'NOTA_QUIMICA', 'NOTA_CFJC', 'NOTA_TEXTOS', 'NOTA_SEMINARIO', 'NOTA_EE_UNO','RENDIMIENTO_DOS'],
        '3': ['PROMEDIO_UNO', 'NAA_UNO', 'NOTA_DIFERENCIAL', 'NOTA_DIBUJO', 'NOTA_TEXTOS', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NCC_DOS', 'NCA_DOS', 'NAA_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_PBASICA', 'NOTA_EE_DOS', 'RENDIMIENTO_TRES'],
        '4': ['PROMEDIO_UNO', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_ESTADISTICA_UNO', 'NOTA_TERMODINAMICA', 'NOTA_TGS', 'NOTA_EE_TRES','RENDIMIENTO_CUATRO'],
        '5': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS','PROMEDIO_TRES', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_TERMODINAMICA', 'NOTA_ECUACIONES', 'NOTA_ESTADISTICA_DOS', 'NOTA_FISICA_DOS', 'NOTA_MECANICA', 'NOTA_PROCESOSQ', 'RENDIMIENTO_CINCO'],
        '6': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_ADMINISTRACION', 'NOTA_LENGUA_UNO', 'NOTA_EI_UNO', 'NOTA_EI_DOS', 'RENDIMIENTO_SEIS'],
        '7': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_LENGUA_UNO', 'NOTA_EI_DOS', 'PROMEDIO_SEIS', 'NCA_SEIS', 'NOTA_PLINEAL', 'NOTA_DISE O', 'NOTA_EI_TRES','RENDIMIENTO_SIETE'],
        '8': ['PROMEDIO_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_LENGUA_UNO','PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'RENDIMIENTO_OCHO'],
        '9': ['PROMEDIO_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'PROMEDIO_OCHO', 'NCC_OCHO', 'NOTA_LOG_UNO', 'NOTA_GOPERACIONES','NOTA_CALIDAD_DOS', 'NOTA_LENGUA_DOS', 'NOTA_CONTEXTO', 'RENDIMIENTO_NUEVE'],
        '10': ['PROMEDIO_SEIS', 'PROMEDIO_SIETE', 'PROMEDIO_OCHO', 'NOTA_CALIDAD_DOS', 'PROMEDIO_NUEVE', 'NAA_NUEVE', 'NOTA_GRADO_UNO', 'NOTA_LOG_DOS', 'NOTA_FINANZAS', 'NOTA_HISTORIA', 'RENDIMIENTO_DIEZ']
    },
    'sistemas': {
        '1': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES', 'QUIMICA_ICFES', 'IDIOMA_ICFES', 'LOCALIDAD', 'RENDIMIENTO_UNO'],
        '2': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'IDIOMA_ICFES', 'PROMEDIO_UNO', 'NOTA_DIFERENCIAL', 'NOTA_PROG_BASICA', 'NOTA_CATEDRA_FJC', 'NOTA_TEXTOS', 'NOTA_SEMINARIO', 'NOTA_CATEDRA_DEM', 'NOTA_CATEDRA_CON', 'NOTA_LOGICA', 'RENDIMIENTO_DOS'],
        '3': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'IDIOMA_ICFES', 'PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_PROG_BASICA', 'NOTA_TEXTOS', 'NOTA_CATEDRA_DEM', 'NOTA_INTEGRAL', 'NOTA_PROG_ORIENTADA', 'NOTA_ETICA', 'RENDIMIENTO_TRES'],
        '4': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'IDIOMA_ICFES', 'PROMEDIO_UNO', 'PROMEDIO_DOS', 'PROMEDIO_TRES', 'NOTA_PROG_BASICA', 'NOTA_TEXTOS', 'NOTA_CATEDRA_DEM', 'NOTA_INTEGRAL', 'NOTA_PROG_ORIENTADA', 'NOTA_ETICA', 'NOTA_FISICA_DOS', 'NOTA_TGS', 'NOTA_PROG_AVANZADA', 'RENDIMIENTO_CUATRO'],
        '5': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_TERMODINAMICA', 'NOTA_ECUACIONES', 'NOTA_ESTADISTICA_DOS', 'NOTA_FISICA_DOS', 'NOTA_MECANICA', 'NOTA_PROCESOSQ', 'RENDIMIENTO_CINCO'],
        '6': ['PROMEDIO_UNO', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_ECONOMIA_UNO', 'NOTA_FISICA_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_ADMINISTRACION', 'NOTA_LENGUA_UNO', 'NOTA_EI_UNO', 'NOTA_EI_DOS', 'RENDIMIENTO_SEIS'],
        '7': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_LENGUA_UNO', 'NOTA_EI_DOS', 'PROMEDIO_SEIS', 'NCA_SEIS', 'NOTA_PLINEAL', 'NOTA_DISE O', 'NOTA_EI_TRES', 'RENDIMIENTO_SIETE'],
        '8': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_LENGUA_UNO', 'PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'RENDIMIENTO_OCHO'],
        '9': ['PROMEDIO_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'PROMEDIO_OCHO', 'NCC_OCHO', 'NOTA_LOG_UNO', 'NOTA_GOPERACIONES', 'NOTA_CALIDAD_DOS', 'NOTA_LENGUA_DOS', 'NOTA_CONTEXTO', 'RENDIMIENTO_NUEVE'],
        '10': ['PROMEDIO_SEIS', 'PROMEDIO_SIETE', 'PROMEDIO_OCHO', 'NOTA_CALIDAD_DOS', 'PROMEDIO_NUEVE', 'NAA_NUEVE', 'NOTA_GRADO_UNO', 'NOTA_LOG_DOS', 'NOTA_FINANZAS', 'NOTA_HISTORIA', 'RENDIMIENTO_DIEZ']
    },
    'catastral': {
        '1': ['variable1_catastral', 'variable2_catastral', 'variable3_catastral'],
        '2': ['variable4_catastral', 'variable5_catastral', 'variable6_catastral']
    }
}

In [6]:
def cargar_datos(carrera, semestre):
    
    ruta_archivo = f'C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/UNIVERSIDAD-DISTRITAL-PROYECTO/MÓDULO ANALÍTICA PREDICTIVA/DATOS/{carrera}{semestre}.csv'
    datos = pd.read_csv(ruta_archivo,sep=";")
    
    return datos
carrera="industrial"
semestre="2"
modelo="CatBoostClassifier"
modelos_seleccionados = ['KNeighbors', 'RedNeuronal']  # Lista de modelos que quieres incluir


In [7]:
datos = cargar_datos(carrera, semestre)
columnas_filtradas = variables_por_carrera[carrera][semestre]
df = datos[columnas_filtradas]
print("DataFrame con columnas filtradas:")
df=df.astype(int)
df

DataFrame con columnas filtradas:


,LOCALIDAD_COLEGIO,PG_ICFES,CON_MAT_ICFES,FISICA_ICFES,BIOLOGIA_ICFES,IDIOMA_ICFES,LOCALIDAD,PROMEDIO_UNO,CAR_UNO,NCC_UNO,NAA_UNO,NOTA_DIFERENCIAL,NOTA_DIBUJO,NOTA_QUIMICA,NOTA_CFJC,NOTA_TEXTOS,NOTA_SEMINARIO,NOTA_EE_UNO,RENDIMIENTO_DOS
0,20,472,65,63,59,62,8,39,0,9,4,35,0,45,40,0,40,0,1
1,0,478,62,58,59,53,8,31,1,19,6,22,30,40,35,30,40,36,2
2,20,465,78,66,49,88,19,30,1,21,5,30,23,35,40,15,40,40,1
3,10,491,72,56,65,53,10,31,1,21,5,20,40,35,40,30,40,0,2
4,20,464,67,64,62,53,20,36,0,15,7,30,35,40,40,35,35,45,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2634,0,342,68,0,68,58,11,31,0,10,4,24,33,0,35,37,43,0,1
2635,0,347,69,0,71,71,19,36,0,15,6,41,28,30,38,39,48,38,1
2636,0,348,67,0,67,68,11,29,0,13,4,25,36,30,30,27,37,0,1
2637,0,340,70,0,67,63,11,31,0,10,4,28,33,0,34,30,39,0,1


In [8]:
def numero_a_letras(numero):
    numeros_letras = ['cero', 'uno', 'dos', 'tres', 'cuatro', 'cinco', 'seis', 'siete', 'ocho', 'nueve', 'diez']
    return numeros_letras[int(numero)]

semestre_en_letras = numero_a_letras(semestre)
print(semestre_en_letras)


dos


In [9]:
X = df.loc[:, ~df.columns.str.contains(f'RENDIMIENTO_{semestre_en_letras.upper()}')]
Y = df.loc[:, df.columns.str.contains(f'RENDIMIENTO_{semestre_en_letras.upper()}')]                                                     
print("Separación de datos usando Pandas") 
print(X.shape, Y.shape)

Separación de datos usando Pandas
(2639, 18) (2639, 1)


In [10]:
#X = X.astype('float32')                         
Y = LabelEncoder().fit_transform(Y.astype('str'))                
print(X.shape, Y.shape)

(2639, 18) (2639,)


In [11]:
from collections import Counter
frecuencias = Counter(Y)
for valor, frecuencia in frecuencias.items():
    print(f"El valor {valor} aparece {frecuencia} veces")

El valor 0 aparece 1121 veces
El valor 1 aparece 1412 veces
El valor 2 aparece 103 veces
El valor 3 aparece 3 veces


### TRANSFORMACIÓN DE DATOS

In [12]:
X_T_JOHNSON1 = X.copy(deep=True)
def transformacion_johnson(X):
    transformador_johnson = PowerTransformer(method='yeo-johnson', standardize=True).fit(X)
    datos_transformados = transformador_johnson.transform(X)
    set_printoptions(precision=3)
    print(datos_transformados[:5, :])
    datos_transformados_df = pd.DataFrame(data=datos_transformados, columns=X.columns)
    return datos_transformados_df
Xpandas_T_JOHNSON1 = transformacion_johnson(X_T_JOHNSON1)
Xpandas_T_JOHNSON1.head(2)

[[ 1.220e+00  1.211e+00 -6.616e-01  7.632e-01 -3.454e-01 -1.764e-03
  -6.866e-01  1.419e+00 -1.149e+00 -2.320e+00 -5.930e-01  9.884e-01
  -2.309e+00  2.011e+00  4.505e-01 -2.261e+00  4.796e-01 -1.137e+00]
 [-1.675e+00  1.339e+00 -9.879e-01  6.658e-01 -3.454e-01 -7.067e-01
  -6.866e-01 -1.514e-01  1.494e-01  2.141e-01  5.492e-01 -5.259e-01
  -2.823e-01  1.378e+00 -3.777e-01 -4.599e-01  4.796e-01  7.941e-01]
 [ 1.220e+00  1.062e+00  7.691e-01  8.197e-01 -1.557e+00  2.110e+00
   1.199e+00 -3.103e-01  1.494e-01  5.505e-01 -5.574e-02  3.774e-01
  -9.903e-01  7.732e-01  4.505e-01 -1.622e+00  4.796e-01  8.704e-01]
 [-2.297e-02  1.622e+00  1.055e-01  6.257e-01  3.520e-01 -7.067e-01
  -3.062e-01 -1.514e-01  1.494e-01  5.505e-01 -5.574e-02 -7.357e-01
   9.297e-01  7.732e-01  4.505e-01 -4.599e-01  4.796e-01 -1.137e+00]
 [ 1.220e+00  1.041e+00 -4.432e-01  7.822e-01  5.774e-03 -7.067e-01
   1.352e+00  7.658e-01 -1.149e+00 -5.846e-01  1.219e+00  3.774e-01
   2.954e-01  1.378e+00  4.505e-01  1.394e-0

,LOCALIDAD_COLEGIO,PG_ICFES,CON_MAT_ICFES,FISICA_ICFES,BIOLOGIA_ICFES,IDIOMA_ICFES,LOCALIDAD,PROMEDIO_UNO,CAR_UNO,NCC_UNO,NAA_UNO,NOTA_DIFERENCIAL,NOTA_DIBUJO,NOTA_QUIMICA,NOTA_CFJC,NOTA_TEXTOS,NOTA_SEMINARIO,NOTA_EE_UNO
0,1.219622,1.210515,-0.661598,0.763182,-0.345439,-0.001764,-0.686598,1.419203,-1.148708,-2.319518,-0.592955,0.988445,-2.309482,2.011394,0.450509,-2.260658,0.479615,-1.136816
1,-1.674845,1.339257,-0.987923,0.665829,-0.345439,-0.706712,-0.686598,-0.151358,0.149437,0.214105,0.549199,-0.525931,-0.282291,1.377582,-0.377682,-0.459859,0.479615,0.794095


In [ ]:
"""
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

# Crear una instancia de RandomOverSampler
ros = RandomOverSampler(sampling_strategy='auto', random_state=42)

# Aplicar RandomOverSampler para generar muestras sintéticas de las clases minoritarias
Xpandas_T_JOHNSON1, Y = ros.fit_resample(Xpandas_T_JOHNSON1, Y)

# Imprimir la distribución de clases después del oversampling
print("Distribución de clases después del oversampling:", Counter(Y))
"""


'\nfrom imblearn.over_sampling import RandomOverSampler\nfrom sklearn.model_selection import train_test_split\n\n# Crear una instancia de RandomOverSampler\nros = RandomOverSampler(sampling_strategy=\'auto\', random_state=42)\n\n# Aplicar RandomOverSampler para generar muestras sintéticas de las clases minoritarias\nXpandas_T_JOHNSON1, Y = ros.fit_resample(Xpandas_T_JOHNSON1, Y)\n\n# Imprimir la distribución de clases después del oversampling\nprint("Distribución de clases después del oversampling:", Counter(Y))\n'

In [13]:
X_trn, X_tst, Y_trn, Y_tst = train_test_split(Xpandas_T_JOHNSON1, Y, test_size=0.3, random_state=2)
print('DATOS: Son {} datos para entrenamiento y {} datos para prueba'.format(X_trn.shape[0], X_tst.shape[0]))

DATOS: Son 1847 datos para entrenamiento y 792 datos para prueba


In [ ]:
"""
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

ros = RandomOverSampler(sampling_strategy='auto', random_state=42)

X_trn,Y_trn = ros.fit_resample(X_trn, Y_trn)

print("Distribución de clases después del oversampling:", Counter(Y_trn))

"""

'\nfrom imblearn.over_sampling import RandomOverSampler\nfrom sklearn.model_selection import train_test_split\n\nros = RandomOverSampler(sampling_strategy=\'auto\', random_state=42)\n\nX_trn,Y_trn = ros.fit_resample(X_trn, Y_trn)\n\nprint("Distribución de clases después del oversampling:", Counter(Y_trn))\n\n'

In [ ]:
from collections import Counter
frecuencias = Counter(Y_trn)   
for valor, frecuencia in frecuencias.items():
    print(f"El valor {valor} aparece {frecuencia} veces")   

El valor 1 aparece 988 veces
El valor 0 aparece 783 veces
El valor 2 aparece 73 veces
El valor 3 aparece 3 veces


### KNEIGHBORSCLASSIFIER

In [ ]:
# conocer los hiperparámetro de cada algoritmo
estimator = KNeighborsClassifier()
estimator.get_params()


{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [ ]:
def entrenar_modelo_knn_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = {
        'n_neighbors': [i for i in range(1, 18, 1)],
        'metric': ['euclidean', 'manhattan', 'minkowski'],
        'algorithm': ['auto', 'kd_tree','ball_tree','brute'],
        #'p': [i for i in range(1, 18,1)],
        'weights': ['uniform'],
        
    }
    modelo = KNeighborsClassifier()
    semilla = 5
    num_folds = 10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn_transformado, Y_trn)
    mejores_hiperparametros_knn=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = KNeighborsClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo ,grid_resultado.best_params_
X_trn = X_trn
Y_trn = Y_trn 
modelo_knn, mejores_hiperparametros_knn = entrenar_modelo_knn_con_transformacion(X_trn, Y_trn)
mejores_hiperparametros_knn

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 82.8933607520564
Mejor valor PARAMETRO usando k-fold: {'algorithm': 'auto', 'metric': 'manhattan', 'n_neighbors': 14, 'weights': 'uniform'}


{'algorithm': 'auto',
 'metric': 'manhattan',
 'n_neighbors': 14,
 'weights': 'uniform'}

In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_knn = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_knn.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_knn_entrenamiento = pd.concat([resultados_df_knn, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_knn_entrenamiento["MODELO"]='KNeighbors'
resultados_df_knn_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_knn_entrenamiento


Precisión:  85.18
Exhaustividad:  84.19
Puntuación F1:  82.48
Exactitud:  84.19
Información Mutua Normalizada (NMI): 43.12
Índice Kappa de Cohen: 68.88


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,85.18,KNeighbors,Entrenamiento
1,Exhaustividad,84.19,KNeighbors,Entrenamiento
2,Puntuación F1,82.48,KNeighbors,Entrenamiento
3,Exactitud,84.19,KNeighbors,Entrenamiento
4,Información Mutua Normalizada (NMI),43.12,KNeighbors,Entrenamiento
5,Índice Kappa de Cohen,68.88,KNeighbors,Entrenamiento


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_knn = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_knn.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_knn_prueba = pd.concat([resultados_df_knn, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_knn_prueba["MODELO"]='KNeighbors'
resultados_df_knn_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_knn_prueba

Precisión:  78.54
Exhaustividad:  80.81
Puntuación F1:  79.18
Exactitud:  80.81
Información Mutua Normalizada (NMI): 33.91
Índice Kappa de Cohen: 62.01


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,78.54,KNeighbors,Prueba
1,Exhaustividad,80.81,KNeighbors,Prueba
2,Puntuación F1,79.18,KNeighbors,Prueba
3,Exactitud,80.81,KNeighbors,Prueba
4,Información Mutua Normalizada (NMI),33.91,KNeighbors,Prueba
5,Índice Kappa de Cohen,62.01,KNeighbors,Prueba


In [ ]:
mejores_hiperparametros_knn = modelo_knn.get_params()
mejores_hiperparametros_knn

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'manhattan',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 14,
 'p': 2,
 'weights': 'uniform'}

In [ ]:
cadena_hiperparametros_knn = ', '.join([f"{key}: {value}" for key, value in mejores_hiperparametros_knn.items()])
df_hiperparametros_knn = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_knn],
    'MODELO': ['KNeighbors'],
    'TIPO_DE_DATOS': ['Entrenamiento']
})
resultados_df_knn = pd.concat([resultados_df_knn_prueba,resultados_df_knn_entrenamiento,df_hiperparametros_knn], ignore_index=True)
resultados_df_knn['TIPO_DE_DATOS']=np.where(resultados_df_knn['MÉTRICA']=='Mejores Hiperparametros','Hiperparametros del modelo',resultados_df_knn['TIPO_DE_DATOS'])
resultados_df_knn

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,78.54,KNeighbors,Prueba
1,Exhaustividad,80.81,KNeighbors,Prueba
2,Puntuación F1,79.18,KNeighbors,Prueba
3,Exactitud,80.81,KNeighbors,Prueba
4,Información Mutua Normalizada (NMI),33.91,KNeighbors,Prueba
5,Índice Kappa de Cohen,62.01,KNeighbors,Prueba
6,Precisión,85.18,KNeighbors,Entrenamiento
7,Exhaustividad,84.19,KNeighbors,Entrenamiento
8,Puntuación F1,82.48,KNeighbors,Entrenamiento
9,Exactitud,84.19,KNeighbors,Entrenamiento


### SVC

In [ ]:
estimator = SVC()
estimator.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [ ]:
def entrenar_modelo_svc_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = { 'kernel':   ['rbf', 'poly', 'sigmoid','linear'], 
                'C': [i/10000 for i in range(8,12,1)],
                'max_iter':[i for i in range(1,3,1)],
                'gamma' : [i/100 for i in range(80,110,5)],
                'random_state':[i for i in range(1,5,1)],
                #'break_ties': [True, False],
                #'cache_size': [100, 200, 300],
                #'class_weight': [None, 'balanced'],
                #'coef0': [0.0, 1.0, 2.0]
                #'decision_function_shape': ['ovo', 'ovr']
                #'degree': [2, 3, 4],
                #'probability': [True, False]
                #'shrinking': [True, False],
                #'tol': [0.0001, 0.001, 0.01],
                #'verbose': [True, False]
                }
    modelo = SVC()
    semilla=5
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_svc=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = SVC(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo ,grid_resultado.best_params_

X_trn = X_trn
Y_trn = Y_trn 

modelo_svc,mejores_hiperparametros_svc = entrenar_modelo_svc_con_transformacion(X_trn, Y_trn)
mejores_hiperparametros_svc

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 58.69271445358402
Mejor valor PARAMETRO usando k-fold: {'C': 0.0008, 'gamma': 0.8, 'kernel': 'linear', 'max_iter': 2, 'random_state': 1}


{'C': 0.0008,
 'gamma': 0.8,
 'kernel': 'linear',
 'max_iter': 2,
 'random_state': 1}

In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_SVC = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_svc.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_svc_entrenamiento = pd.concat([resultados_df_SVC, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_svc_entrenamiento["MODELO"]='SVC'
resultados_df_svc_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_svc_entrenamiento

Precisión:  65.41
Exhaustividad:  60.31
Puntuación F1:  61.56
Exactitud:  60.31
Información Mutua Normalizada (NMI): 15.06
Índice Kappa de Cohen: 31.43


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,65.41,SVC,Entrenamiento
1,Exhaustividad,60.31,SVC,Entrenamiento
2,Puntuación F1,61.56,SVC,Entrenamiento
3,Exactitud,60.31,SVC,Entrenamiento
4,Información Mutua Normalizada (NMI),15.06,SVC,Entrenamiento
5,Índice Kappa de Cohen,31.43,SVC,Entrenamiento


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_SVC = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_svc.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_svc_prueba = pd.concat([resultados_df_SVC, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_svc_prueba["MODELO"]='SVC'
resultados_df_svc_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_svc_prueba

Precisión:  64.67
Exhaustividad:  60.23
Puntuación F1:  61.1
Exactitud:  60.23
Información Mutua Normalizada (NMI): 13.71
Índice Kappa de Cohen: 30.68


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,64.67,SVC,Prueba
1,Exhaustividad,60.23,SVC,Prueba
2,Puntuación F1,61.10,SVC,Prueba
3,Exactitud,60.23,SVC,Prueba
4,Información Mutua Normalizada (NMI),13.71,SVC,Prueba
5,Índice Kappa de Cohen,30.68,SVC,Prueba


In [ ]:
mejores_hiperparametros_svc = modelo_svc.get_params()
mejores_hiperparametros_svc

{'C': 0.0008,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 0.8,
 'kernel': 'linear',
 'max_iter': 2,
 'probability': False,
 'random_state': 1,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [ ]:
cadena_hiperparametros_svc = ', '.join([f"{key}: {value}" for key, value in mejores_hiperparametros_svc.items()])
df_hiperparametros_svc = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_svc],
    'MODELO': ['SVC'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})
resultados_df_svc = pd.concat([resultados_df_svc_prueba,resultados_df_svc_entrenamiento,df_hiperparametros_svc], ignore_index=True)
resultados_df_svc

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,64.67,SVC,Prueba
1,Exhaustividad,60.23,SVC,Prueba
2,Puntuación F1,61.1,SVC,Prueba
3,Exactitud,60.23,SVC,Prueba
4,Información Mutua Normalizada (NMI),13.71,SVC,Prueba
5,Índice Kappa de Cohen,30.68,SVC,Prueba
6,Precisión,65.41,SVC,Entrenamiento
7,Exhaustividad,60.31,SVC,Entrenamiento
8,Puntuación F1,61.56,SVC,Entrenamiento
9,Exactitud,60.31,SVC,Entrenamiento


### DECISION TREE

In [15]:
def entrenar_modelo_tree_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = {          
              'max_depth':[i for i in range(1,7,1)],
              'min_samples_split' :  [i for i in range(1,7,1)],  
              'min_samples_leaf' : [i for i in range(1,7,1)], 
              'max_features' : [i for i in range(1,7,1)], 
              'splitter': ["best", "random"],
              'random_state': [i for i in range(1,5,1)],
              #'criterion': ['gini','entropy','log_loss']
            
}
    modelo = DecisionTreeClassifier()
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_tree=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = DecisionTreeClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo,grid_resultado.best_params_

X_trn = X_trn
Y_trn = Y_trn 

modelo_tree,mejores_hiperparametros_tree = entrenar_modelo_tree_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 82.18595769682727
Mejor valor PARAMETRO usando k-fold: {'max_depth': 6, 'max_features': 3, 'min_samples_leaf': 6, 'min_samples_split': 2, 'random_state': 1, 'splitter': 'best'}


In [16]:
# Predecir las etiquetas para los datos de prueba
resultados_df_tree = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_tree.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_tree_entrenamiento = pd.concat([resultados_df_tree, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_tree_entrenamiento["MODELO"]='DecisionTree'
resultados_df_tree_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_tree_entrenamiento

Precisión:  80.31
Exhaustividad:  83.54
Puntuación F1:  81.78
Exactitud:  83.54
Información Mutua Normalizada (NMI): 40.92
Índice Kappa de Cohen: 67.73


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.31,DecisionTree,Entrenamiento
1,Exhaustividad,83.54,DecisionTree,Entrenamiento
2,Puntuación F1,81.78,DecisionTree,Entrenamiento
3,Exactitud,83.54,DecisionTree,Entrenamiento
4,Información Mutua Normalizada (NMI),40.92,DecisionTree,Entrenamiento
5,Índice Kappa de Cohen,67.73,DecisionTree,Entrenamiento


In [17]:
# Predecir las etiquetas para los datos de prueba
resultados_df_tree = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_tree.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_tree_prueba = pd.concat([resultados_df_tree, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_tree_prueba["MODELO"]='DecisionTree'
resultados_df_tree_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_tree_prueba

Precisión:  77.12
Exhaustividad:  80.05
Puntuación F1:  78.47
Exactitud:  80.05
Información Mutua Normalizada (NMI): 31.68
Índice Kappa de Cohen: 60.83


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,77.12,DecisionTree,Prueba
1,Exhaustividad,80.05,DecisionTree,Prueba
2,Puntuación F1,78.47,DecisionTree,Prueba
3,Exactitud,80.05,DecisionTree,Prueba
4,Información Mutua Normalizada (NMI),31.68,DecisionTree,Prueba
5,Índice Kappa de Cohen,60.83,DecisionTree,Prueba


In [18]:
mejores_hiperparametros_tree = modelo_tree.get_params()
mejores_hiperparametros_tree

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 6,
 'max_features': 3,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 6,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'random_state': 1,
 'splitter': 'best'}

In [19]:
cadena_hiperparametros_tree = ', '.join([f"{key}: {value}" for key, value in mejores_hiperparametros_tree.items()])
df_hiperparametros_tree = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_tree],
    'MODELO': ['DecisionTree'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})
resultados_df_tree = pd.concat([resultados_df_tree_prueba,resultados_df_tree_entrenamiento,df_hiperparametros_tree], ignore_index=True)
resultados_df_tree

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,77.12,DecisionTree,Prueba
1,Exhaustividad,80.05,DecisionTree,Prueba
2,Puntuación F1,78.47,DecisionTree,Prueba
3,Exactitud,80.05,DecisionTree,Prueba
4,Información Mutua Normalizada (NMI),31.68,DecisionTree,Prueba
5,Índice Kappa de Cohen,60.83,DecisionTree,Prueba
6,Precisión,80.31,DecisionTree,Entrenamiento
7,Exhaustividad,83.54,DecisionTree,Entrenamiento
8,Puntuación F1,81.78,DecisionTree,Entrenamiento
9,Exactitud,83.54,DecisionTree,Entrenamiento


### NAIVE BAYES

In [ ]:
# conocer los hiperparámetro de cada algoritmo
estimator =  GaussianNB()
estimator.get_params()


{'priors': None, 'var_smoothing': 1e-09}

In [ ]:
def entrenar_modelo_gaussian_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = {}
    modelo = GaussianNB()
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = GaussianNB(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_gaussian = entrenar_modelo_gaussian_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 73.08901292596946
Mejor valor PARAMETRO usando k-fold: {}


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_gaussian = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_gaussian.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_gaussian_entrenamiento = pd.concat([resultados_df_gaussian, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_gaussian_entrenamiento["MODELO"]='NaiveBayes'
resultados_df_gaussian_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_gaussian_entrenamiento

Precisión:  79.28
Exhaustividad:  73.2
Puntuación F1:  75.51
Exactitud:  73.2
Información Mutua Normalizada (NMI): 32.62
Índice Kappa de Cohen: 53.62


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.28,NaiveBayes,Entrenamiento
1,Exhaustividad,73.20,NaiveBayes,Entrenamiento
2,Puntuación F1,75.51,NaiveBayes,Entrenamiento
3,Exactitud,73.20,NaiveBayes,Entrenamiento
4,Información Mutua Normalizada (NMI),32.62,NaiveBayes,Entrenamiento
5,Índice Kappa de Cohen,53.62,NaiveBayes,Entrenamiento


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_gaussian = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_gaussian.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_gaussian_prueba = pd.concat([resultados_df_gaussian, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_gaussian_prueba["MODELO"]='NaiveBayes'
resultados_df_gaussian_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_gaussian_prueba

Precisión:  79.91
Exhaustividad:  73.74
Puntuación F1:  76.22
Exactitud:  73.74
Información Mutua Normalizada (NMI): 31.25
Índice Kappa de Cohen: 54.1


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.91,NaiveBayes,Prueba
1,Exhaustividad,73.74,NaiveBayes,Prueba
2,Puntuación F1,76.22,NaiveBayes,Prueba
3,Exactitud,73.74,NaiveBayes,Prueba
4,Información Mutua Normalizada (NMI),31.25,NaiveBayes,Prueba
5,Índice Kappa de Cohen,54.10,NaiveBayes,Prueba


In [ ]:
cadena_hiperparametros_gaussian = 'No tiene hiperparametros'
df_hiperparametros_gaussian = pd.DataFrame({
    'MÉTRICA': ['No tiene Hiperparametros'],
    'VALOR': [cadena_hiperparametros_gaussian],
    'MODELO': ['NaiveBayes'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})

In [ ]:
resultados_df_gaussian = pd.concat([resultados_df_gaussian_prueba,resultados_df_gaussian_entrenamiento,df_hiperparametros_gaussian], ignore_index=True)
resultados_df_gaussian

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.91,NaiveBayes,Prueba
1,Exhaustividad,73.74,NaiveBayes,Prueba
2,Puntuación F1,76.22,NaiveBayes,Prueba
3,Exactitud,73.74,NaiveBayes,Prueba
4,Información Mutua Normalizada (NMI),31.25,NaiveBayes,Prueba
5,Índice Kappa de Cohen,54.1,NaiveBayes,Prueba
6,Precisión,79.28,NaiveBayes,Entrenamiento
7,Exhaustividad,73.2,NaiveBayes,Entrenamiento
8,Puntuación F1,75.51,NaiveBayes,Entrenamiento
9,Exactitud,73.2,NaiveBayes,Entrenamiento


### LDA

In [ ]:
# conocer los hiperparámetro de cada algoritmo
estimator = LinearDiscriminantAnalysis()
estimator.get_params()

{'covariance_estimator': None,
 'n_components': None,
 'priors': None,
 'shrinkage': None,
 'solver': 'svd',
 'store_covariance': False,
 'tol': 0.0001}

In [ ]:
def entrenar_modelo_LDA_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = { 'solver':  ['svd','lsqr','eigen'],
            'n_components':[1,2,3,4,5,6,7,8,9,10],
            'shrinkage': ['auto', 0.001, 0.01, 0.1, 0.5,1,10,100,1000],
            'tol':[i/1000 for i in range(1,100,1)]}
    modelo = LinearDiscriminantAnalysis()
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_LDA=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = LinearDiscriminantAnalysis(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo, grid_resultado.best_params_

X_trn = X_trn
Y_trn = Y_trn 

modelo_LDA,mejores_hiperparametros_LDA = entrenar_modelo_LDA_con_transformacion(X_trn, Y_trn)
mejores_hiperparametros_LDA


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 80.12602820211517
Mejor valor PARAMETRO usando k-fold: {'n_components': 1, 'shrinkage': 0.001, 'solver': 'lsqr', 'tol': 0.001}


{'n_components': 1, 'shrinkage': 0.001, 'solver': 'lsqr', 'tol': 0.001}

In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_LDA = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_LDA.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_LDA_entrenamiento = pd.concat([resultados_df_LDA, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_LDA_entrenamiento["MODELO"]='LDA'
resultados_df_LDA_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_LDA_entrenamiento

Precisión:  79.01
Exhaustividad:  80.4
Puntuación F1:  79.3
Exactitud:  80.4
Información Mutua Normalizada (NMI): 33.78
Índice Kappa de Cohen: 61.93


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.01,LDA,Entrenamiento
1,Exhaustividad,80.40,LDA,Entrenamiento
2,Puntuación F1,79.30,LDA,Entrenamiento
3,Exactitud,80.40,LDA,Entrenamiento
4,Información Mutua Normalizada (NMI),33.78,LDA,Entrenamiento
5,Índice Kappa de Cohen,61.93,LDA,Entrenamiento


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_LDA = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_LDA.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_LDA_prueba = pd.concat([resultados_df_LDA, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_LDA_prueba["MODELO"]='LDA'
resultados_df_LDA_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_LDA_prueba

Precisión:  79.76
Exhaustividad:  79.92
Puntuación F1:  78.67
Exactitud:  79.92
Información Mutua Normalizada (NMI): 31.79
Índice Kappa de Cohen: 60.55


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.76,LDA,Prueba
1,Exhaustividad,79.92,LDA,Prueba
2,Puntuación F1,78.67,LDA,Prueba
3,Exactitud,79.92,LDA,Prueba
4,Información Mutua Normalizada (NMI),31.79,LDA,Prueba
5,Índice Kappa de Cohen,60.55,LDA,Prueba


In [ ]:
mejores_hiperparametros_LDA = modelo_LDA.get_params()
mejores_hiperparametros_LDA

{'covariance_estimator': None,
 'n_components': 1,
 'priors': None,
 'shrinkage': 0.001,
 'solver': 'lsqr',
 'store_covariance': False,
 'tol': 0.001}

In [ ]:
cadena_hiperparametros_LDA = ', '.join([f"{key}: {value}" for key, value in mejores_hiperparametros_LDA.items()])
df_hiperparametros_LDA = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_LDA],
    'MODELO': ['LDA'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})

resultados_df_LDA = pd.concat([resultados_df_LDA_prueba,resultados_df_LDA_entrenamiento,df_hiperparametros_LDA], ignore_index=True)
resultados_df_LDA

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.76,LDA,Prueba
1,Exhaustividad,79.92,LDA,Prueba
2,Puntuación F1,78.67,LDA,Prueba
3,Exactitud,79.92,LDA,Prueba
4,Información Mutua Normalizada (NMI),31.79,LDA,Prueba
5,Índice Kappa de Cohen,60.55,LDA,Prueba
6,Precisión,79.01,LDA,Entrenamiento
7,Exhaustividad,80.4,LDA,Entrenamiento
8,Puntuación F1,79.3,LDA,Entrenamiento
9,Exactitud,80.4,LDA,Entrenamiento


### BAGGINGCLASSIFIER

In [20]:
from sklearn.ensemble import BaggingClassifier
estimator = BaggingClassifier()
estimator.get_params()

{'bootstrap': True,
 'bootstrap_features': False,
 'estimator': None,
 'max_features': 1.0,
 'max_samples': 1.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [21]:
def entrenar_modelo_BG_con_transformacion(X_trn, Y_trn,mejores_hiperparametros_tree):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'n_estimators': [i for i in range(750,760,5)],
            'max_samples' : [i/100.0 for i in range(70,90,5)],
            'max_features': [i/100.0 for i in range(75,85,5)],
            'bootstrap': [True], 
            'bootstrap_features': [True]}
    base_estimator= DecisionTreeClassifier(**mejores_hiperparametros_tree)
    semilla=7
    modelo = BaggingClassifier(estimator=base_estimator,n_estimators=750, random_state=semilla,
                             bootstrap= True, bootstrap_features = True, max_features = 0.7,
                             max_samples= 0.5)
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_BG=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = BaggingClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo,grid_resultado.best_params_
X_trn = X_trn
Y_trn = Y_trn 
modelo_BG,mejores_hiperparametros_BG = entrenar_modelo_BG_con_transformacion(X_trn, Y_trn,mejores_hiperparametros_tree)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 83.75558166862514
Mejor valor PARAMETRO usando k-fold: {'bootstrap': True, 'bootstrap_features': True, 'max_features': 0.8, 'max_samples': 0.75, 'n_estimators': 750}


In [22]:
# Predecir las etiquetas para los datos de prueba
resultados_df_BG = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_BG.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_BG_entrenamiento = pd.concat([resultados_df_BG, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_BG_entrenamiento["MODELO"]='Bagging'
resultados_df_BG_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_BG_entrenamiento

Precisión:  100.0
Exhaustividad:  100.0
Puntuación F1:  100.0
Exactitud:  100.0
Información Mutua Normalizada (NMI): 100.0
Índice Kappa de Cohen: 100.0


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,100.0,Bagging,Entrenamiento
1,Exhaustividad,100.0,Bagging,Entrenamiento
2,Puntuación F1,100.0,Bagging,Entrenamiento
3,Exactitud,100.0,Bagging,Entrenamiento
4,Información Mutua Normalizada (NMI),100.0,Bagging,Entrenamiento
5,Índice Kappa de Cohen,100.0,Bagging,Entrenamiento


In [23]:
# Predecir las etiquetas para los datos de prueba
resultados_df_BG = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_BG.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_BG_prueba = pd.concat([resultados_df_BG, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_BG_prueba["MODELO"]='Bagging'
resultados_df_BG_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_BG_prueba

Precisión:  82.55
Exhaustividad:  82.95
Puntuación F1:  81.7
Exactitud:  82.95
Información Mutua Normalizada (NMI): 38.09
Índice Kappa de Cohen: 66.62


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,82.55,Bagging,Prueba
1,Exhaustividad,82.95,Bagging,Prueba
2,Puntuación F1,81.70,Bagging,Prueba
3,Exactitud,82.95,Bagging,Prueba
4,Información Mutua Normalizada (NMI),38.09,Bagging,Prueba
5,Índice Kappa de Cohen,66.62,Bagging,Prueba


In [24]:
mejores_hiperparametros_BG = modelo_BG.get_params()
mejores_hiperparametros_BG

{'bootstrap': True,
 'bootstrap_features': True,
 'estimator': None,
 'max_features': 0.8,
 'max_samples': 0.75,
 'n_estimators': 750,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [25]:
cadena_hiperparametros_BG = ', '.join([f"{key}: {value}" for key, value in mejores_hiperparametros_BG.items()])
df_hiperparametros_BG = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_BG],
    'MODELO': ['Bagging'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})
resultados_df_BG = pd.concat([resultados_df_BG_prueba,resultados_df_BG_entrenamiento,df_hiperparametros_BG], ignore_index=True)
resultados_df_BG

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,82.55,Bagging,Prueba
1,Exhaustividad,82.95,Bagging,Prueba
2,Puntuación F1,81.7,Bagging,Prueba
3,Exactitud,82.95,Bagging,Prueba
4,Información Mutua Normalizada (NMI),38.09,Bagging,Prueba
5,Índice Kappa de Cohen,66.62,Bagging,Prueba
6,Precisión,100.0,Bagging,Entrenamiento
7,Exhaustividad,100.0,Bagging,Entrenamiento
8,Puntuación F1,100.0,Bagging,Entrenamiento
9,Exactitud,100.0,Bagging,Entrenamiento


### RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier
estimator = RandomForestClassifier()
estimator.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [36]:
def entrenar_modelo_random_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = { 
                'min_samples_split' : [1, 2 , 3,  4 , 6 , 8 , 10 , 15, 20 ],  
                'min_samples_leaf' : [ 1 , 3 , 5 , 7 , 9, 12, 15 ],
                'criterion':('gini','entropy','log_loss'),
                #'max_features':('sqrt','log2',None)
                #'class_weight':('balanced','balanced_subsample')
              }
    modelo = RandomForestClassifier()
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_random=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = RandomForestClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo,grid_resultado.best_params_

X_trn = X_trn
Y_trn = Y_trn 

modelo_random,mejores_hiperparametros_random = entrenar_modelo_random_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 84.78319623971797
Mejor valor PARAMETRO usando k-fold: {'criterion': 'log_loss', 'min_samples_leaf': 5, 'min_samples_split': 6}


In [37]:
# Predecir las etiquetas para los datos de prueba
resultados_df_random = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_random.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_random_entrenamiento = pd.concat([resultados_df_random, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_random_entrenamiento["MODELO"]='RandomForest'
resultados_df_random_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_random_entrenamiento

Precisión:  92.01
Exhaustividad:  91.72
Puntuación F1:  90.59
Exactitud:  91.72
Información Mutua Normalizada (NMI): 65.54
Índice Kappa de Cohen: 83.89


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,92.01,RandomForest,Entrenamiento
1,Exhaustividad,91.72,RandomForest,Entrenamiento
2,Puntuación F1,90.59,RandomForest,Entrenamiento
3,Exactitud,91.72,RandomForest,Entrenamiento
4,Información Mutua Normalizada (NMI),65.54,RandomForest,Entrenamiento
5,Índice Kappa de Cohen,83.89,RandomForest,Entrenamiento


In [38]:
# Predecir las etiquetas para los datos de prueba
resultados_df_random = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_random.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_random_prueba = pd.concat([resultados_df_random, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_random_prueba["MODELO"]='RandomForest'
resultados_df_random_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_random_prueba

Precisión:  80.03
Exhaustividad:  83.08
Puntuación F1:  81.46
Exactitud:  83.08
Información Mutua Normalizada (NMI): 38.62
Índice Kappa de Cohen: 66.82


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.03,RandomForest,Prueba
1,Exhaustividad,83.08,RandomForest,Prueba
2,Puntuación F1,81.46,RandomForest,Prueba
3,Exactitud,83.08,RandomForest,Prueba
4,Información Mutua Normalizada (NMI),38.62,RandomForest,Prueba
5,Índice Kappa de Cohen,66.82,RandomForest,Prueba


In [39]:
mejores_hiperparametros_random = modelo_random.get_params()
mejores_hiperparametros_random

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'log_loss',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 5,
 'min_samples_split': 6,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:
cadena_hiperparametros_random = ', '.join([f"{key}: {value}" for key, value in mejores_hiperparametros_random.items()])
df_hiperparametros_random = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_random],
    'MODELO': ['RandomForest'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})

resultados_df_random = pd.concat([resultados_df_random_prueba,resultados_df_random_entrenamiento,df_hiperparametros_random], ignore_index=True)
resultados_df_random

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,84.46,RandomForest,Prueba
1,Exhaustividad,83.84,RandomForest,Prueba
2,Puntuación F1,82.39,RandomForest,Prueba
3,Exactitud,83.84,RandomForest,Prueba
4,Información Mutua Normalizada (NMI),40.5,RandomForest,Prueba
5,Índice Kappa de Cohen,68.42,RandomForest,Prueba
6,Precisión,93.68,RandomForest,Entrenamiento
7,Exhaustividad,93.56,RandomForest,Entrenamiento
8,Puntuación F1,92.93,RandomForest,Entrenamiento
9,Exactitud,93.56,RandomForest,Entrenamiento


### EXTRA TREES CLASSIFIER

In [32]:
def entrenar_modelo_extra_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'min_samples_split' : [i for i in range(1,10,1)], 
                'min_samples_leaf' : [i for i in range(0,10,1)],
                #'max_features':[i for i in range(0,5,1)],
                #'max_depth':[i for i in range(0,5,1)],
                'min_samples_leaf':[i for i in range(0,10,1)],
                'min_samples_split':[i for i in range(0,10,1)],
                'criterion':('gini','entropy','log_loss')}
    
    semilla=7            
    modelo = ExtraTreesClassifier(random_state=semilla, 
                                n_estimators=40,
                                bootstrap=True) 
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_extra=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = ExtraTreesClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo,grid_resultado.best_params_

X_trn = X_trn
Y_trn = Y_trn 

modelo_extra,mejores_hiperparametros_extra = entrenar_modelo_extra_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 84.78349001175089
Mejor valor PARAMETRO usando k-fold: {'criterion': 'gini', 'min_samples_leaf': 1, 'min_samples_split': 9}


In [33]:
# Predecir las etiquetas para los datos de prueba
resultados_df_extra = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_extra.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_extra_entrenamiento = pd.concat([resultados_df_extra, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_extra_entrenamiento["MODELO"]='ExtraTrees'
resultados_df_extra_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_extra_entrenamiento


Precisión:  96.72
Exhaustividad:  96.59
Puntuación F1:  96.4
Exactitud:  96.59
Información Mutua Normalizada (NMI): 82.97
Índice Kappa de Cohen: 93.48


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,96.72,ExtraTrees,Entrenamiento
1,Exhaustividad,96.59,ExtraTrees,Entrenamiento
2,Puntuación F1,96.40,ExtraTrees,Entrenamiento
3,Exactitud,96.59,ExtraTrees,Entrenamiento
4,Información Mutua Normalizada (NMI),82.97,ExtraTrees,Entrenamiento
5,Índice Kappa de Cohen,93.48,ExtraTrees,Entrenamiento


In [34]:
# Predecir las etiquetas para los datos de prueba
resultados_df_extra = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_extra.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_extra_prueba = pd.concat([resultados_df_extra, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_extra_prueba["MODELO"]='ExtraTrees'
resultados_df_extra_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_extra_prueba

Precisión:  81.0
Exhaustividad:  84.09
Puntuación F1:  82.46
Exactitud:  84.09
Información Mutua Normalizada (NMI): 41.62
Índice Kappa de Cohen: 68.81


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,81.00,ExtraTrees,Prueba
1,Exhaustividad,84.09,ExtraTrees,Prueba
2,Puntuación F1,82.46,ExtraTrees,Prueba
3,Exactitud,84.09,ExtraTrees,Prueba
4,Información Mutua Normalizada (NMI),41.62,ExtraTrees,Prueba
5,Índice Kappa de Cohen,68.81,ExtraTrees,Prueba


In [35]:
mejores_hiperparametros_extra = modelo_extra.get_params()
mejores_hiperparametros_extra

{'bootstrap': False,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 9,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:
cadena_hiperparametros_extra = ', '.join([f"{key}: {value}" for key, value in mejores_hiperparametros_extra.items()])
df_hiperparametros_extra = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_extra],
    'MODELO': ['ExtraTrees'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})


resultados_df_extra = pd.concat([resultados_df_extra_prueba,resultados_df_extra_entrenamiento,df_hiperparametros_extra], ignore_index=True)
resultados_df_extra

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,84.33,ExtraTrees,Prueba
1,Exhaustividad,83.59,ExtraTrees,Prueba
2,Puntuación F1,82.13,ExtraTrees,Prueba
3,Exactitud,83.59,ExtraTrees,Prueba
4,Información Mutua Normalizada (NMI),39.78,ExtraTrees,Prueba
5,Índice Kappa de Cohen,67.8,ExtraTrees,Prueba
6,Precisión,96.24,ExtraTrees,Entrenamiento
7,Exhaustividad,96.1,ExtraTrees,Entrenamiento
8,Puntuación F1,95.87,ExtraTrees,Entrenamiento
9,Exactitud,96.1,ExtraTrees,Entrenamiento


### ADABOOST

In [ ]:
from sklearn.ensemble import AdaBoostClassifier            
estimator  = AdaBoostClassifier ()
estimator.get_params()

{'algorithm': 'SAMME.R',
 'estimator': None,
 'learning_rate': 1.0,
 'n_estimators': 50,
 'random_state': None}

In [ ]:
def entrenar_modelo_ADA_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'learning_rate' : [i/10000.0 for i in range(5,20,5)],
                  'n_estimators':[i for i in range(1,50,1)]}
    semilla=7            
    modelo = AdaBoostClassifier(estimator = None,  algorithm = 'SAMME.R', 
                                random_state= None) 
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_ADA=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = AdaBoostClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo,grid_resultado.best_params_

X_trn = X_trn
Y_trn = Y_trn 

modelo_ADA, mejores_hiperparametros_ADA= entrenar_modelo_ADA_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 77.52996474735605
Mejor valor PARAMETRO usando k-fold: {'learning_rate': 0.0005, 'n_estimators': 46}


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_ADA = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_ADA.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_ADA_entrenamiento = pd.concat([resultados_df_ADA, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_ADA_entrenamiento["MODELO"]='AdaBoost'
resultados_df_ADA_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_ADA_entrenamiento

Precisión:  75.68
Exhaustividad:  78.13
Puntuación F1:  76.32
Exactitud:  78.13
Información Mutua Normalizada (NMI): 29.41
Índice Kappa de Cohen: 56.68


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,75.68,AdaBoost,Entrenamiento
1,Exhaustividad,78.13,AdaBoost,Entrenamiento
2,Puntuación F1,76.32,AdaBoost,Entrenamiento
3,Exactitud,78.13,AdaBoost,Entrenamiento
4,Información Mutua Normalizada (NMI),29.41,AdaBoost,Entrenamiento
5,Índice Kappa de Cohen,56.68,AdaBoost,Entrenamiento


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_ADA = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_ADA.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_ADA_prueba = pd.concat([resultados_df_ADA, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_ADA_prueba["MODELO"]='AdaBoost'
resultados_df_ADA_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_ADA_prueba

Precisión:  76.39
Exhaustividad:  78.28
Puntuación F1:  76.53
Exactitud:  78.28
Información Mutua Normalizada (NMI): 29.74
Índice Kappa de Cohen: 56.76


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,76.39,AdaBoost,Prueba
1,Exhaustividad,78.28,AdaBoost,Prueba
2,Puntuación F1,76.53,AdaBoost,Prueba
3,Exactitud,78.28,AdaBoost,Prueba
4,Información Mutua Normalizada (NMI),29.74,AdaBoost,Prueba
5,Índice Kappa de Cohen,56.76,AdaBoost,Prueba


In [ ]:
mejores_hiperparametros_ADA = modelo_ADA.get_params()
mejores_hiperparametros_ADA

{'algorithm': 'SAMME.R',
 'estimator': None,
 'learning_rate': 0.0005,
 'n_estimators': 46,
 'random_state': None}

In [ ]:
cadena_hiperparametros_ADA = ', '.join([f"{key}: {value}" for key, value in mejores_hiperparametros_ADA.items()])
df_hiperparametros_ADA = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_ADA],
    'MODELO': ['AdaBoost'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})

resultados_df_ADA = pd.concat([resultados_df_ADA_prueba,resultados_df_ADA_entrenamiento,df_hiperparametros_ADA], ignore_index=True)
resultados_df_ADA

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,76.39,AdaBoost,Prueba
1,Exhaustividad,78.28,AdaBoost,Prueba
2,Puntuación F1,76.53,AdaBoost,Prueba
3,Exactitud,78.28,AdaBoost,Prueba
4,Información Mutua Normalizada (NMI),29.74,AdaBoost,Prueba
5,Índice Kappa de Cohen,56.76,AdaBoost,Prueba
6,Precisión,75.68,AdaBoost,Entrenamiento
7,Exhaustividad,78.13,AdaBoost,Entrenamiento
8,Puntuación F1,76.32,AdaBoost,Entrenamiento
9,Exactitud,78.13,AdaBoost,Entrenamiento


### GRADIENT BOOSTING MACHINE

In [26]:
def entrenar_modelo_GD_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = { 
                'learning_rate' : [0.01, 0.05, 0.1,0.15],
                'n_estimators': [i for i in range(100,1200,100)],
                'loss':('log_loss','exponential'),
                'criterion':['friedman_mse']     
              }
    semilla=7
    modelo = GradientBoostingClassifier(random_state=semilla,
                                    n_estimators= 100,learning_rate= 0.1,max_depth= 2,
                                    min_samples_split= 2, min_samples_leaf= 3,max_features= 2)
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_GD=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = GradientBoostingClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo,grid_resultado.best_params_

X_trn = X_trn
Y_trn = Y_trn 

modelo_GD,mejores_hiperparametros_GD = entrenar_modelo_GD_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 83.75440658049355
Mejor valor PARAMETRO usando k-fold: {'criterion': 'friedman_mse', 'learning_rate': 0.01, 'loss': 'log_loss', 'n_estimators': 800}


In [27]:
# Predecir las etiquetas para los datos de prueba
resultados_df_GD = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_GD.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_GD_entrenamiento = pd.concat([resultados_df_GD, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_GD_entrenamiento["MODELO"]='GradientBoosting'
resultados_df_GD_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_GD_entrenamiento

Precisión:  92.36
Exhaustividad:  92.15
Puntuación F1:  91.9
Exactitud:  92.15
Información Mutua Normalizada (NMI): 65.43
Índice Kappa de Cohen: 84.96


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,92.36,GradientBoosting,Entrenamiento
1,Exhaustividad,92.15,GradientBoosting,Entrenamiento
2,Puntuación F1,91.90,GradientBoosting,Entrenamiento
3,Exactitud,92.15,GradientBoosting,Entrenamiento
4,Información Mutua Normalizada (NMI),65.43,GradientBoosting,Entrenamiento
5,Índice Kappa de Cohen,84.96,GradientBoosting,Entrenamiento


In [28]:
# Predecir las etiquetas para los datos de prueba
resultados_df_GD = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_GD.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_GD_prueba = pd.concat([resultados_df_GD, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_GD_prueba["MODELO"]='GradientBoosting'
resultados_df_GD_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_GD_prueba

Precisión:  81.24
Exhaustividad:  81.94
Puntuación F1:  81.04
Exactitud:  81.94
Información Mutua Normalizada (NMI): 36.25
Índice Kappa de Cohen: 64.96


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,81.24,GradientBoosting,Prueba
1,Exhaustividad,81.94,GradientBoosting,Prueba
2,Puntuación F1,81.04,GradientBoosting,Prueba
3,Exactitud,81.94,GradientBoosting,Prueba
4,Información Mutua Normalizada (NMI),36.25,GradientBoosting,Prueba
5,Índice Kappa de Cohen,64.96,GradientBoosting,Prueba


In [29]:
mejores_hiperparametros_GD = modelo_GD.get_params()
mejores_hiperparametros_GD

{'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.01,
 'loss': 'log_loss',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 800,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [30]:
cadena_hiperparametros_GD = ', '.join([f"{key}: {value}" for key, value in mejores_hiperparametros_GD.items()])
df_hiperparametros_GD = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_GD],
    'MODELO': ['GradientBoosting'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})
resultados_df_GD = pd.concat([resultados_df_GD_prueba,resultados_df_GD_entrenamiento,df_hiperparametros_GD], ignore_index=True)
resultados_df_GD

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,81.24,GradientBoosting,Prueba
1,Exhaustividad,81.94,GradientBoosting,Prueba
2,Puntuación F1,81.04,GradientBoosting,Prueba
3,Exactitud,81.94,GradientBoosting,Prueba
4,Información Mutua Normalizada (NMI),36.25,GradientBoosting,Prueba
5,Índice Kappa de Cohen,64.96,GradientBoosting,Prueba
6,Precisión,92.36,GradientBoosting,Entrenamiento
7,Exhaustividad,92.15,GradientBoosting,Entrenamiento
8,Puntuación F1,91.9,GradientBoosting,Entrenamiento
9,Exactitud,92.15,GradientBoosting,Entrenamiento


### XGBOOST

In [ ]:
def entrenar_modelo_XB_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'reg_alpha': [0,0.1,0.2,0.3,0.4,0.5],
                'reg_lambda':  [i/1000.0 for i in range(100,150,5)],
                'n_estimators':  [i for i in range(1,10,2)],
                'colsample_bytree': [0.1,0.3, 0.5,0.6,0.7,0.8, 0.9, 1,1.1],
                'objective' : ('binary:logistic', 'Multi: softprob'),
                'loss': ['log_loss'],
                'max_features':('sqrt','log2')
                }
    semilla=7
    modelo = XGBClassifier(random_state=semilla,subsample =1,max_depth =2)
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_XB=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = XGBClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo,grid_resultado.best_params_

X_trn = X_trn
Y_trn = Y_trn 

modelo_XB,mejores_hiperparametros_XB = entrenar_modelo_XB_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 83.26880141010575
Mejor valor PARAMETRO usando k-fold: {'colsample_bytree': 1, 'loss': 'log_loss', 'max_features': 'sqrt', 'n_estimators': 9, 'objective': 'binary:logistic', 'reg_alpha': 0, 'reg_lambda': 0.1}


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_XB = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_XB.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_XB_entrenamiento = pd.concat([resultados_df_XB, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_XB_entrenamiento["MODELO"]=' XGB'
resultados_df_XB_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_XB_entrenamiento

Precisión:  93.81
Exhaustividad:  93.56
Puntuación F1:  93.23
Exactitud:  93.56
Información Mutua Normalizada (NMI): 70.53
Índice Kappa de Cohen: 87.62


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,93.81,XGB,Entrenamiento
1,Exhaustividad,93.56,XGB,Entrenamiento
2,Puntuación F1,93.23,XGB,Entrenamiento
3,Exactitud,93.56,XGB,Entrenamiento
4,Información Mutua Normalizada (NMI),70.53,XGB,Entrenamiento
5,Índice Kappa de Cohen,87.62,XGB,Entrenamiento


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_XB = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_XB.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_XB_prueba = pd.concat([resultados_df_XB, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_XB_prueba["MODELO"]='XGB'
resultados_df_XB_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_XB_prueba

Precisión:  79.48
Exhaustividad:  82.45
Puntuación F1:  80.9
Exactitud:  82.45
Información Mutua Normalizada (NMI): 37.24
Índice Kappa de Cohen: 65.66


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.48,XGB,Prueba
1,Exhaustividad,82.45,XGB,Prueba
2,Puntuación F1,80.90,XGB,Prueba
3,Exactitud,82.45,XGB,Prueba
4,Información Mutua Normalizada (NMI),37.24,XGB,Prueba
5,Índice Kappa de Cohen,65.66,XGB,Prueba


In [ ]:
mejores_hiperparametros_XB = modelo_XB.get_params()
mejores_hiperparametros_XB

{'objective': 'multi:softprob',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': 1,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': 9,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': 0,
 'reg_lambda': 0.1,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None,
 'loss': 'log_loss',
 'max_features': 'sqrt'}

In [ ]:
cadena_hiperparametros_XB = ', '.join([f"{key}: {value}" for key, value in mejores_hiperparametros_XB.items()])
df_hiperparametros_XB = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_XB],
    'MODELO': ['XGB'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})

resultados_df_XB = pd.concat([resultados_df_XB_prueba,resultados_df_XB_entrenamiento,df_hiperparametros_XB], ignore_index=True)
resultados_df_XB

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.48,XGB,Prueba
1,Exhaustividad,82.45,XGB,Prueba
2,Puntuación F1,80.9,XGB,Prueba
3,Exactitud,82.45,XGB,Prueba
4,Información Mutua Normalizada (NMI),37.24,XGB,Prueba
5,Índice Kappa de Cohen,65.66,XGB,Prueba
6,Precisión,93.81,XGB,Entrenamiento
7,Exhaustividad,93.56,XGB,Entrenamiento
8,Puntuación F1,93.23,XGB,Entrenamiento
9,Exactitud,93.56,XGB,Entrenamiento


### CATBOOST

In [ ]:
estimator  = CatBoostClassifier ()
estimator.get_params()

{}

In [ ]:
def entrenar_modelo_CB_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'border_count':[53],'l2_leaf_reg': [42],'learning_rate': [0.01],
                  'depth': [4, 6, 8],'thread_count': [4, 8, 12]
                  } 
    semilla=7
    modelo = CatBoostClassifier(random_state=semilla, verbose =0)
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_CB=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = CatBoostClassifier(verbose=0,**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo,grid_resultado.best_params_

X_trn = X_trn
Y_trn = Y_trn 

modelo_CB,mejores_hiperparametros_CB = entrenar_modelo_CB_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 83.75470035252644
Mejor valor PARAMETRO usando k-fold: {'border_count': 53, 'depth': 4, 'l2_leaf_reg': 42, 'learning_rate': 0.01, 'thread_count': 4}


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_CB = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_CB.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento.ravel())
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_CB_entrenamiento = pd.concat([resultados_df_CB, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_CB_entrenamiento["MODELO"]='CatBoost'
resultados_df_CB_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_CB_entrenamiento

Precisión:  81.66
Exhaustividad:  84.95
Puntuación F1:  83.17
Exactitud:  84.95
Información Mutua Normalizada (NMI): 44.96
Índice Kappa de Cohen: 70.51


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,81.66,CatBoost,Entrenamiento
1,Exhaustividad,84.95,CatBoost,Entrenamiento
2,Puntuación F1,83.17,CatBoost,Entrenamiento
3,Exactitud,84.95,CatBoost,Entrenamiento
4,Información Mutua Normalizada (NMI),44.96,CatBoost,Entrenamiento
5,Índice Kappa de Cohen,70.51,CatBoost,Entrenamiento


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_CB = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_CB.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba.ravel())
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_CB_prueba = pd.concat([resultados_df_CB, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_CB_prueba["MODELO"]='CatBoost'
resultados_df_CB_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_CB_prueba

Precisión:  80.04
Exhaustividad:  83.08
Puntuación F1:  81.46
Exactitud:  83.08
Información Mutua Normalizada (NMI): 38.61
Índice Kappa de Cohen: 66.8


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.04,CatBoost,Prueba
1,Exhaustividad,83.08,CatBoost,Prueba
2,Puntuación F1,81.46,CatBoost,Prueba
3,Exactitud,83.08,CatBoost,Prueba
4,Información Mutua Normalizada (NMI),38.61,CatBoost,Prueba
5,Índice Kappa de Cohen,66.80,CatBoost,Prueba


In [ ]:
mejores_hiperparametros_CB = modelo_CB.get_params()
mejores_hiperparametros_CB

{'learning_rate': 0.01,
 'depth': 4,
 'l2_leaf_reg': 42,
 'border_count': 53,
 'thread_count': 4,
 'verbose': 0}

In [ ]:
cadena_hiperparametros_CB = ', '.join([f"{key}: {value}" for key, value in mejores_hiperparametros_CB.items()])
df_hiperparametros_CB = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_CB],
    'MODELO': ['CatBoost'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})

resultados_df_CB = pd.concat([resultados_df_CB_prueba,resultados_df_CB_entrenamiento,df_hiperparametros_CB], ignore_index=True)
resultados_df_CB

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.04,CatBoost,Prueba
1,Exhaustividad,83.08,CatBoost,Prueba
2,Puntuación F1,81.46,CatBoost,Prueba
3,Exactitud,83.08,CatBoost,Prueba
4,Información Mutua Normalizada (NMI),38.61,CatBoost,Prueba
5,Índice Kappa de Cohen,66.8,CatBoost,Prueba
6,Precisión,81.66,CatBoost,Entrenamiento
7,Exhaustividad,84.95,CatBoost,Entrenamiento
8,Puntuación F1,83.17,CatBoost,Entrenamiento
9,Exactitud,84.95,CatBoost,Entrenamiento


### LIGHT GBM

In [ ]:
def entrenar_modelo_LIGHT_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {
    'min_child_samples' : [i for i in range(200, 10000, 100)],'colsample_bytree': [0.6, 0.8, 1.0],
    'boosting_type': ['gbdt', 'dart', 'goss'],'objective': ['binary', 'multiclass'],'random_state': [42]}
    semilla=7
    modelo = LGBMClassifier (random_state=semilla,                           
                            num_leaves =  10,max_depth = 1, n_estimators = 100,    
                            learning_rate = 0.1 ,class_weight=  None, subsample = 1,
                            colsample_bytree= 1, reg_alpha=  0, reg_lambda = 0,
                            min_split_gain = 0, boosting_type = 'gbdt')
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_LIGHT=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = LGBMClassifier(verbose=-1,**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo,grid_resultado.best_params_

X_trn = X_trn
Y_trn = Y_trn 

modelo_LIGHT,mejores_hiperparametros_LIGHT = entrenar_modelo_LIGHT_con_transformacion(X_trn, Y_trn)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000426 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 859
[LightGBM] [Info] Number of data points in the train set: 1847, number of used features: 18
[LightGBM] [Info] Start training from score -0.858185
[LightGBM] [Info] Start training from score -0.625635
[LightGBM] [Info] Start training from score -3.230859
[LightGBM] [Info] Start training from score -6.422706
Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 83.64747356051706
Mejor valor PARAMETRO usando k-fold: {'boosting_type': 'gbdt', 'colsample_bytree': 1.0, 'min_child_samples': 300, 'objective': 'multiclass', 'random_state': 42}


In [ ]:
resultados_df_LIGHT = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_LIGHT.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_LIGHT_entrenamiento = pd.concat([resultados_df_LIGHT, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_LIGHT_entrenamiento["MODELO"]='LGBM'
resultados_df_LIGHT_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_LIGHT_entrenamiento

Precisión:  88.92
Exhaustividad:  88.47
Puntuación F1:  87.6
Exactitud:  88.47
Información Mutua Normalizada (NMI): 54.84
Índice Kappa de Cohen: 77.66


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,88.92,LGBM,Entrenamiento
1,Exhaustividad,88.47,LGBM,Entrenamiento
2,Puntuación F1,87.60,LGBM,Entrenamiento
3,Exactitud,88.47,LGBM,Entrenamiento
4,Información Mutua Normalizada (NMI),54.84,LGBM,Entrenamiento
5,Índice Kappa de Cohen,77.66,LGBM,Entrenamiento


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_LIGHT = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_LIGHT.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_LIGHT_prueba = pd.concat([resultados_df_LIGHT, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_LIGHT_prueba["MODELO"]='LGBM'
resultados_df_LIGHT_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_LIGHT_prueba

Precisión:  82.11
Exhaustividad:  82.83
Puntuación F1:  81.77
Exactitud:  82.83
Información Mutua Normalizada (NMI): 37.95
Índice Kappa de Cohen: 66.66


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,82.11,LGBM,Prueba
1,Exhaustividad,82.83,LGBM,Prueba
2,Puntuación F1,81.77,LGBM,Prueba
3,Exactitud,82.83,LGBM,Prueba
4,Información Mutua Normalizada (NMI),37.95,LGBM,Prueba
5,Índice Kappa de Cohen,66.66,LGBM,Prueba


In [ ]:
mejores_hiperparametros_LIGHT = modelo_LIGHT.get_params()
mejores_hiperparametros_LIGHT

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 300,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'num_leaves': 31,
 'objective': 'multiclass',
 'random_state': 42,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0,
 'verbose': -1}

In [ ]:
cadena_hiperparametros_LIGHT = ', '.join([f"{key}: {value}" for key, value in mejores_hiperparametros_LIGHT.items()])
df_hiperparametros_LIGHT = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_LIGHT],
    'MODELO': ['LGBM'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})

resultados_df_LIGHT = pd.concat([resultados_df_LIGHT_prueba,resultados_df_LIGHT_entrenamiento,df_hiperparametros_LIGHT], ignore_index=True)
resultados_df_LIGHT

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,82.11,LGBM,Prueba
1,Exhaustividad,82.83,LGBM,Prueba
2,Puntuación F1,81.77,LGBM,Prueba
3,Exactitud,82.83,LGBM,Prueba
4,Información Mutua Normalizada (NMI),37.95,LGBM,Prueba
5,Índice Kappa de Cohen,66.66,LGBM,Prueba
6,Precisión,88.92,LGBM,Entrenamiento
7,Exhaustividad,88.47,LGBM,Entrenamiento
8,Puntuación F1,87.6,LGBM,Entrenamiento
9,Exactitud,88.47,LGBM,Entrenamiento


### VOTING - VOTACIÓN DURA

In [ ]:
def entrenar_modelo_voting_hard_con_transformacion(X_trn, Y_trn,
                                                   mejores_hiperparametros_GD,
                                                   mejores_hiperparametros_tree,
                                                   mejores_hiperparametros_ADA,
                                                   mejores_hiperparametros_extra,
                                                   mejores_hiperparametros_random,
                                                   mejores_hiperparametros_BG,
                                                   mejores_hiperparametros_XB):
    X_trn_transformado = X_trn
    semilla= 7 
    kfold = StratifiedKFold(n_splits=10, random_state=semilla, shuffle=True)
    modelo1 = GradientBoostingClassifier(**mejores_hiperparametros_GD)
    base_estimator=DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo2 = AdaBoostClassifier(**mejores_hiperparametros_ADA)
    modelo3 = ExtraTreesClassifier(**mejores_hiperparametros_extra)
    modelo4 = RandomForestClassifier (**mejores_hiperparametros_random)
    model = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo5 = BaggingClassifier(**mejores_hiperparametros_BG)
    modelo6 = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo7 = XGBClassifier(**mejores_hiperparametros_XB)
    metrica = 'accuracy'
    mejor_modelo = VotingClassifier(
    estimators=[('Gradient', modelo1), ('Adaboost', modelo2), 
                                    ('Extratrees', modelo3),('Random Forest',modelo4),
                                    ('Bagging',modelo5),('Decision tree',modelo6),
                                    ('XGB',modelo7)],voting='hard') 
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    resultados = cross_val_score(mejor_modelo, X_trn_transformado, Y_trn, 
                                    cv=kfold,scoring = metrica)
    mejores_hiperparametros_voting_hard=mejor_modelo.get_params
    print("Rendimiento del modelo:") 
    print ("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean()*100," % ") 
    return mejor_modelo, mejores_hiperparametros_voting_hard
modelo_voting_hard, mejores_hiperparametros_voting_hard= entrenar_modelo_voting_hard_con_transformacion(X_trn, Y_trn,
                                                   mejores_hiperparametros_GD,
                                                   mejores_hiperparametros_tree,
                                                   mejores_hiperparametros_ADA,
                                                   mejores_hiperparametros_extra,
                                                   mejores_hiperparametros_random,
                                                   mejores_hiperparametros_BG,
                                                   mejores_hiperparametros_XB)

Rendimiento del modelo:
Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold: 83.91774383078732  % 


In [ ]:
resultados_df_voting_hard = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_voting_hard.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_voting_hard_entrenamiento = pd.concat([resultados_df_voting_hard, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_voting_hard_entrenamiento["MODELO"]='VotingHard'
resultados_df_voting_hard_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_voting_hard_entrenamiento

Precisión:  94.86
Exhaustividad:  94.64
Puntuación F1:  94.29
Exactitud:  94.64
Información Mutua Normalizada (NMI): 75.06
Índice Kappa de Cohen: 89.7


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,94.86,VotingHard,Entrenamiento
1,Exhaustividad,94.64,VotingHard,Entrenamiento
2,Puntuación F1,94.29,VotingHard,Entrenamiento
3,Exactitud,94.64,VotingHard,Entrenamiento
4,Información Mutua Normalizada (NMI),75.06,VotingHard,Entrenamiento
5,Índice Kappa de Cohen,89.70,VotingHard,Entrenamiento


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_voting_hard = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_voting_hard.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_voting_hard_prueba = pd.concat([resultados_df_voting_hard, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_voting_hard_prueba["MODELO"]='VotingHard'
resultados_df_voting_hard_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_voting_hard_prueba

Precisión:  80.04
Exhaustividad:  83.08
Puntuación F1:  81.46
Exactitud:  83.08
Información Mutua Normalizada (NMI): 38.61
Índice Kappa de Cohen: 66.8


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.04,VotingHard,Prueba
1,Exhaustividad,83.08,VotingHard,Prueba
2,Puntuación F1,81.46,VotingHard,Prueba
3,Exactitud,83.08,VotingHard,Prueba
4,Información Mutua Normalizada (NMI),38.61,VotingHard,Prueba
5,Índice Kappa de Cohen,66.80,VotingHard,Prueba


In [ ]:
estimadores = modelo_voting_hard.estimators
hiperparametros_voting_hard = []
for nombre, estimador in estimadores:
    hiperparametros = estimador.get_params()
    hiperparametros_estimador = {'Estimador': nombre, 'Hiperparametros': hiperparametros}
    hiperparametros_voting_hard.append(hiperparametros_estimador)

hiperparametros_voting_hard

[{'Estimador': 'Gradient',
  'Hiperparametros': {'ccp_alpha': 0.0,
   'criterion': 'friedman_mse',
   'init': None,
   'learning_rate': 0.01,
   'loss': 'log_loss',
   'max_depth': 3,
   'max_features': None,
   'max_leaf_nodes': None,
   'min_impurity_decrease': 0.0,
   'min_samples_leaf': 1,
   'min_samples_split': 2,
   'min_weight_fraction_leaf': 0.0,
   'n_estimators': 800,
   'n_iter_no_change': None,
   'random_state': None,
   'subsample': 1.0,
   'tol': 0.0001,
   'validation_fraction': 0.1,
   'verbose': 0,
   'warm_start': False}},
 {'Estimador': 'Adaboost',
  'Hiperparametros': {'algorithm': 'SAMME.R',
   'estimator': None,
   'learning_rate': 0.0005,
   'n_estimators': 46,
   'random_state': None}},
 {'Estimador': 'Extratrees',
  'Hiperparametros': {'bootstrap': False,
   'ccp_alpha': 0.0,
   'class_weight': None,
   'criterion': 'gini',
   'max_depth': None,
   'max_features': 'sqrt',
   'max_leaf_nodes': None,
   'max_samples': None,
   'min_impurity_decrease': 0.0,
   '

In [ ]:
cadena_hiperparametros_voting_hard = ', '.join([', '.join([f"{key}: {value}" for key, value in estimador.items()]) for estimador in hiperparametros_voting_hard])

df_hiperparametros_voting_hard = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_voting_hard],
    'MODELO': ['VotingHard'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})


resultados_df_voting_hard = pd.concat([resultados_df_voting_hard_prueba,resultados_df_voting_hard_entrenamiento,df_hiperparametros_voting_hard], ignore_index=True)
resultados_df_voting_hard

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.04,VotingHard,Prueba
1,Exhaustividad,83.08,VotingHard,Prueba
2,Puntuación F1,81.46,VotingHard,Prueba
3,Exactitud,83.08,VotingHard,Prueba
4,Información Mutua Normalizada (NMI),38.61,VotingHard,Prueba
5,Índice Kappa de Cohen,66.8,VotingHard,Prueba
6,Precisión,94.86,VotingHard,Entrenamiento
7,Exhaustividad,94.64,VotingHard,Entrenamiento
8,Puntuación F1,94.29,VotingHard,Entrenamiento
9,Exactitud,94.64,VotingHard,Entrenamiento


### VOTING - VOTACIÓN SUAVE

In [ ]:
def entrenar_modelo_voting_soft_con_transformacion(X_trn, Y_trn,
                                                   mejores_hiperparametros_GD,
                                                   mejores_hiperparametros_tree,
                                                   mejores_hiperparametros_ADA,
                                                   mejores_hiperparametros_extra,
                                                   mejores_hiperparametros_random,
                                                   mejores_hiperparametros_BG):
    X_trn_transformado = X_trn
    semilla= 7 
    kfold = StratifiedKFold(n_splits=10, random_state=semilla, shuffle=True)
    modelo1 = GradientBoostingClassifier(**mejores_hiperparametros_GD)
    base_estimator=DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo2 = AdaBoostClassifier(**mejores_hiperparametros_ADA)
    modelo3 = ExtraTreesClassifier(**mejores_hiperparametros_extra)
    modelo4 = RandomForestClassifier (**mejores_hiperparametros_random)
    model = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo5 = BaggingClassifier(**mejores_hiperparametros_BG)
    modelo6 = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    metrica = 'accuracy'
    mejor_modelo = VotingClassifier(
    estimators=[('Gradient', modelo1), ('Adaboost', modelo2), ('Extratrees', modelo3),
                                    ('Random Forest',modelo4),
                                    ('Bagging',modelo5),('Decision tree',modelo6)],
    voting='soft',weights=[0.9,0.7,0.9,0.9,0.9,0.9]) 
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    mejores_hiperparametros_voting_soft=mejor_modelo.get_params
    resultados = cross_val_score(mejor_modelo, X_trn_transformado, Y_trn, cv=kfold,scoring = metrica)
    print("Rendimiento del modelo:") 
    print ("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean()*100," % ") 
    return mejor_modelo,mejores_hiperparametros_voting_soft

# Cargar los datos
#datos = cargar_datos(carrera, semestre)
X_trn = X_trn
Y_trn = Y_trn 

# Entrenar el modelo KNN con transformación Yeo-Johnson
modelo_voting_soft,mejores_hiperparametros_voting_soft = entrenar_modelo_voting_soft_con_transformacion(X_trn, Y_trn,
                                                   mejores_hiperparametros_GD,
                                                   mejores_hiperparametros_tree,
                                                   mejores_hiperparametros_ADA,
                                                   mejores_hiperparametros_extra,
                                                   mejores_hiperparametros_random,
                                                   mejores_hiperparametros_BG)

Rendimiento del modelo:
Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold: 83.80934195064629  % 


In [ ]:
resultados_df_voting_soft = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_voting_soft.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_voting_soft_entrenamiento = pd.concat([resultados_df_voting_soft, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_voting_soft_entrenamiento["MODELO"]='VotingSoft'
resultados_df_voting_soft_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_voting_soft_entrenamiento

Precisión:  92.34
Exhaustividad:  92.1
Puntuación F1:  90.96
Exactitud:  92.1
Información Mutua Normalizada (NMI): 66.94
Índice Kappa de Cohen: 84.64


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,92.34,VotingSoft,Entrenamiento
1,Exhaustividad,92.10,VotingSoft,Entrenamiento
2,Puntuación F1,90.96,VotingSoft,Entrenamiento
3,Exactitud,92.10,VotingSoft,Entrenamiento
4,Información Mutua Normalizada (NMI),66.94,VotingSoft,Entrenamiento
5,Índice Kappa de Cohen,84.64,VotingSoft,Entrenamiento


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_voting_soft = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_voting_soft.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_voting_soft_prueba = pd.concat([resultados_df_voting_soft, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_voting_soft_prueba["MODELO"]='VotingSoft'
resultados_df_voting_soft_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_voting_soft_prueba

Precisión:  80.2
Exhaustividad:  83.21
Puntuación F1:  81.58
Exactitud:  83.21
Información Mutua Normalizada (NMI): 38.92
Índice Kappa de Cohen: 67.02


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.20,VotingSoft,Prueba
1,Exhaustividad,83.21,VotingSoft,Prueba
2,Puntuación F1,81.58,VotingSoft,Prueba
3,Exactitud,83.21,VotingSoft,Prueba
4,Información Mutua Normalizada (NMI),38.92,VotingSoft,Prueba
5,Índice Kappa de Cohen,67.02,VotingSoft,Prueba


In [ ]:
estimadores = modelo_voting_soft.estimators
hiperparametros_voting_soft = []
for nombre, estimador in estimadores:
    hiperparametros = estimador.get_params()
    hiperparametros_estimador = {'Estimador': nombre, 'Hiperparametros': hiperparametros}
    hiperparametros_voting_soft.append(hiperparametros_estimador)



In [ ]:
cadena_hiperparametros_voting_soft = ', '.join([', '.join([f"{key}: {value}" for key, value in estimador.items()]) for estimador in hiperparametros_voting_soft])

df_hiperparametros_voting_soft = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_voting_soft],
    'MODELO': ['VotingSoft'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})


resultados_df_voting_soft = pd.concat([resultados_df_voting_soft_prueba,resultados_df_voting_soft_entrenamiento,df_hiperparametros_voting_soft], ignore_index=True)
resultados_df_voting_soft

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.2,VotingSoft,Prueba
1,Exhaustividad,83.21,VotingSoft,Prueba
2,Puntuación F1,81.58,VotingSoft,Prueba
3,Exactitud,83.21,VotingSoft,Prueba
4,Información Mutua Normalizada (NMI),38.92,VotingSoft,Prueba
5,Índice Kappa de Cohen,67.02,VotingSoft,Prueba
6,Precisión,92.34,VotingSoft,Entrenamiento
7,Exhaustividad,92.1,VotingSoft,Entrenamiento
8,Puntuación F1,90.96,VotingSoft,Entrenamiento
9,Exactitud,92.1,VotingSoft,Entrenamiento


### STACKING ( METAMODELO LINEAL)

In [ ]:
def entrenar_modelo_stacking_lineal_con_transformacion(X_trn, Y_trn,
                                                   mejores_hiperparametros_tree,
                                                   mejores_hiperparametros_ADA,
                                                   mejores_hiperparametros_extra,
                                                   mejores_hiperparametros_random,
                                                   mejores_hiperparametros_BG):
    X_trn_transformado = X_trn
    semilla= 7 
    kfold = StratifiedKFold(n_splits=10, random_state=semilla, shuffle=True)
    base_estimator=DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo2 = AdaBoostClassifier(**mejores_hiperparametros_ADA)
    modelo3 = ExtraTreesClassifier(**mejores_hiperparametros_extra)
    modelo4 = RandomForestClassifier (**mejores_hiperparametros_random)
    model = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo5 = BaggingClassifier(**mejores_hiperparametros_BG)
    modelo6 = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    estimador_final = LogisticRegression()
    metrica = 'accuracy'
    mejor_modelo = StackingClassifier(
    estimators=[ ('Adaboost', modelo2), ('Extratrees', modelo3),('Random Forest',modelo4),
                                    ('Bagging',modelo5),('Decision tree',modelo6)], 
                                    final_estimator=estimador_final) 
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    mejores_hiperparametros_stacking_lineal=mejor_modelo.get_params
    resultados = cross_val_score(mejor_modelo, X_trn_transformado, Y_trn, cv=kfold,scoring = metrica)
    print("Rendimiento del modelo:") 
    print ("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean()*100," % ") 
    return mejor_modelo,mejores_hiperparametros_stacking_lineal

X_trn = X_trn
Y_trn = Y_trn 

modelo_stacking_lineal,mejores_hiperparametros_stacking_lineal = entrenar_modelo_stacking_lineal_con_transformacion(X_trn, Y_trn,
                                                   mejores_hiperparametros_tree,
                                                   mejores_hiperparametros_ADA,
                                                   mejores_hiperparametros_extra,
                                                   mejores_hiperparametros_random,
                                                   mejores_hiperparametros_BG)

Rendimiento del modelo:
Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold: 84.07990599294945  % 


In [ ]:
resultados_df_stacking_lineal = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_stacking_lineal.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_stacking_lineal_entrenamiento = pd.concat([resultados_df_stacking_lineal, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_stacking_lineal_entrenamiento["MODELO"]='StackingLineal'
resultados_df_stacking_lineal_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_stacking_lineal_entrenamiento

Precisión:  98.81
Exhaustividad:  98.97
Puntuación F1:  98.88
Exactitud:  98.97
Información Mutua Normalizada (NMI): 93.6
Índice Kappa de Cohen: 98.06


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,98.81,StackingLineal,Entrenamiento
1,Exhaustividad,98.97,StackingLineal,Entrenamiento
2,Puntuación F1,98.88,StackingLineal,Entrenamiento
3,Exactitud,98.97,StackingLineal,Entrenamiento
4,Información Mutua Normalizada (NMI),93.60,StackingLineal,Entrenamiento
5,Índice Kappa de Cohen,98.06,StackingLineal,Entrenamiento


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_stacking_lineal = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_stacking_lineal.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_stacking_lineal_prueba = pd.concat([resultados_df_stacking_lineal, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_stacking_lineal_prueba["MODELO"]='StackingLineal'
resultados_df_stacking_lineal_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_stacking_lineal_prueba

Precisión:  83.07
Exhaustividad:  83.59
Puntuación F1:  82.33
Exactitud:  83.59
Información Mutua Normalizada (NMI): 39.63
Índice Kappa de Cohen: 67.95


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,83.07,StackingLineal,Prueba
1,Exhaustividad,83.59,StackingLineal,Prueba
2,Puntuación F1,82.33,StackingLineal,Prueba
3,Exactitud,83.59,StackingLineal,Prueba
4,Información Mutua Normalizada (NMI),39.63,StackingLineal,Prueba
5,Índice Kappa de Cohen,67.95,StackingLineal,Prueba


In [ ]:
estimadores = modelo_stacking_lineal.estimators
hiperparametros_stacking_lineal = []
for nombre, estimador in estimadores:
    hiperparametros = estimador.get_params()
    hiperparametros_estimador = {'Estimador': nombre, 'Hiperparametros': hiperparametros}
    hiperparametros_stacking_lineal.append(hiperparametros_estimador)



In [ ]:
cadena_hiperparametros_stacking_lineal = ', '.join([', '.join([f"{key}: {value}" for key, value in estimador.items()]) for estimador in hiperparametros_stacking_lineal])

df_hiperparametros_stacking_lineal = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_stacking_lineal],
    'MODELO': ['StackingLineal'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})


resultados_df_stacking_lineal = pd.concat([resultados_df_stacking_lineal_prueba,resultados_df_stacking_lineal_entrenamiento,df_hiperparametros_stacking_lineal], ignore_index=True)
resultados_df_stacking_lineal

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,83.07,StackingLineal,Prueba
1,Exhaustividad,83.59,StackingLineal,Prueba
2,Puntuación F1,82.33,StackingLineal,Prueba
3,Exactitud,83.59,StackingLineal,Prueba
4,Información Mutua Normalizada (NMI),39.63,StackingLineal,Prueba
5,Índice Kappa de Cohen,67.95,StackingLineal,Prueba
6,Precisión,98.81,StackingLineal,Entrenamiento
7,Exhaustividad,98.97,StackingLineal,Entrenamiento
8,Puntuación F1,98.88,StackingLineal,Entrenamiento
9,Exactitud,98.97,StackingLineal,Entrenamiento


### STACKING (METAMODELO NO LINEAL)

In [ ]:
def entrenar_modelo_stacking_nolineal_con_transformacion(X_trn, Y_trn,
                                                   mejores_hiperparametros_tree,
                                                   mejores_hiperparametros_extra,
                                                   mejores_hiperparametros_random,
                                                   mejores_hiperparametros_BG):
    X_trn_transformado = X_trn
    semilla= 7 
    kfold = StratifiedKFold(n_splits=10, random_state=semilla, shuffle=True)
    modelo1 = ExtraTreesClassifier(**mejores_hiperparametros_extra)
    modelo2 = RandomForestClassifier (**mejores_hiperparametros_random)
    model = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo3 = BaggingClassifier(**mejores_hiperparametros_BG)
    modelo4 = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    estimador_final = ExtraTreesClassifier(**mejores_hiperparametros_extra)
    metrica = 'accuracy'
    mejor_modelo = StackingClassifier(
    estimators=[  ('Extratrees', modelo1),('Random Forest',modelo2),('Bagging',modelo3),
                                    ('Decision tree',modelo4)], 
                                    final_estimator=estimador_final) 
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    mejores_hiperparametros_stacking_nolineal=mejor_modelo.get_params
    resultados = cross_val_score(mejor_modelo, X_trn_transformado, Y_trn, cv=kfold,scoring = metrica)
    print("Rendimiento del modelo:") 
    print ("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean()*100," % ") 
    return mejor_modelo,mejores_hiperparametros_stacking_nolineal
X_trn = X_trn
Y_trn = Y_trn 
modelo_stacking_nolineal,mejores_hiperparametros_stacking_nolineal = entrenar_modelo_stacking_nolineal_con_transformacion(X_trn, Y_trn,
                                                   mejores_hiperparametros_tree,
                                                   mejores_hiperparametros_extra,
                                                   mejores_hiperparametros_random,
                                                   mejores_hiperparametros_BG)

Rendimiento del modelo:
Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold: 83.2141598119859  % 


In [ ]:
resultados_df_stacking_nolineal = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_stacking_nolineal.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_stacking_nolineal_entrenamiento = pd.concat([resultados_df_stacking_nolineal, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_stacking_nolineal_entrenamiento["MODELO"]='Stackingnolineal'
resultados_df_stacking_nolineal_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_stacking_nolineal_entrenamiento

Precisión:  92.31
Exhaustividad:  93.83
Puntuación F1:  92.19
Exactitud:  93.83
Información Mutua Normalizada (NMI): 76.55
Índice Kappa de Cohen: 87.95


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,92.31,Stackingnolineal,Entrenamiento
1,Exhaustividad,93.83,Stackingnolineal,Entrenamiento
2,Puntuación F1,92.19,Stackingnolineal,Entrenamiento
3,Exactitud,93.83,Stackingnolineal,Entrenamiento
4,Información Mutua Normalizada (NMI),76.55,Stackingnolineal,Entrenamiento
5,Índice Kappa de Cohen,87.95,Stackingnolineal,Entrenamiento


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_stacking_nolineal = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])
Y_pred_prueba = modelo_stacking_nolineal.predict(X_tst)
precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_stacking_nolineal_prueba = pd.concat([resultados_df_stacking_nolineal, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_stacking_nolineal_prueba["MODELO"]='Stackingnolineal'
resultados_df_stacking_nolineal_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_stacking_nolineal_prueba

Precisión:  81.43
Exhaustividad:  82.7
Puntuación F1:  81.75
Exactitud:  82.7
Información Mutua Normalizada (NMI): 38.4
Índice Kappa de Cohen: 66.55


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,81.43,Stackingnolineal,Prueba
1,Exhaustividad,82.70,Stackingnolineal,Prueba
2,Puntuación F1,81.75,Stackingnolineal,Prueba
3,Exactitud,82.70,Stackingnolineal,Prueba
4,Información Mutua Normalizada (NMI),38.40,Stackingnolineal,Prueba
5,Índice Kappa de Cohen,66.55,Stackingnolineal,Prueba


In [ ]:
estimadores = modelo_stacking_nolineal.estimators
hiperparametros_stacking_nolineal = []
for nombre, estimador in estimadores:
    hiperparametros = estimador.get_params()
    hiperparametros_estimador = {'Estimador': nombre, 'Hiperparametros': hiperparametros}
    hiperparametros_stacking_nolineal.append(hiperparametros_estimador)


In [ ]:
cadena_hiperparametros_stacking_nolineal = ', '.join([', '.join([f"{key}: {value}" for key, value in estimador.items()]) for estimador in hiperparametros_stacking_nolineal])

df_hiperparametros_stacking_nolineal = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_stacking_nolineal],
    'MODELO': ['Stackingnolineal'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})
resultados_df_stacking_nolineal = pd.concat([resultados_df_stacking_nolineal_prueba,resultados_df_stacking_nolineal_entrenamiento,df_hiperparametros_stacking_nolineal], ignore_index=True)
resultados_df_stacking_nolineal

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,81.43,Stackingnolineal,Prueba
1,Exhaustividad,82.7,Stackingnolineal,Prueba
2,Puntuación F1,81.75,Stackingnolineal,Prueba
3,Exactitud,82.7,Stackingnolineal,Prueba
4,Información Mutua Normalizada (NMI),38.4,Stackingnolineal,Prueba
5,Índice Kappa de Cohen,66.55,Stackingnolineal,Prueba
6,Precisión,92.31,Stackingnolineal,Entrenamiento
7,Exhaustividad,93.83,Stackingnolineal,Entrenamiento
8,Puntuación F1,92.19,Stackingnolineal,Entrenamiento
9,Exactitud,93.83,Stackingnolineal,Entrenamiento


### VOTING WEIGHTED VOTING 

In [ ]:
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

def weighted_voting_ensemble(X_trn, Y_trn, mejores_hiperparametros_GD,
                            mejores_hiperparametros_tree,mejores_hiperparametros_ADA,
                            mejores_hiperparametros_extra,mejores_hiperparametros_random,
                            mejores_hiperparametros_BG,mejores_hiperparametros_XB):
    X_trn_transformado = X_trn
    modelos = [
        GradientBoostingClassifier(**mejores_hiperparametros_GD),
        AdaBoostClassifier(**mejores_hiperparametros_ADA),
        ExtraTreesClassifier(**mejores_hiperparametros_extra),
        RandomForestClassifier(**mejores_hiperparametros_random),
        BaggingClassifier(**mejores_hiperparametros_BG),
        DecisionTreeClassifier(**mejores_hiperparametros_tree),
        XGBClassifier(**mejores_hiperparametros_XB)
    ]
    pesos = np.ones(len(modelos))
    precisión_modelos = []
    for modelo in modelos:
        modelo.fit(X_trn_transformado, Y_trn)
        precisión = np.mean(cross_val_score(modelo, X_trn_transformado, Y_trn, cv=KFold(n_splits=10, random_state=7, shuffle=True), scoring='accuracy'))
        precisión_modelos.append(precisión)
    suma_precisión = sum(precisión_modelos)
    for i in range(len(precisión_modelos)):
        pesos[i] = suma_precisión / (precisión_modelos[i] * len(modelos))

    return modelos, pesos

def weighted_voting_predict(modelos, pesos, X_test):
    predicciones_modelos = [modelo.predict(X_test) for modelo in modelos]
    pesos_float64 = np.array(pesos, dtype=np.float64)
    predicciones_ponderadas = np.zeros_like(predicciones_modelos[0], dtype=np.float64)
    for i, predicciones_modelo in enumerate(predicciones_modelos):
        predicciones_ponderadas += predicciones_modelo * pesos_float64[i]
    predicciones_ponderadas /= len(modelos)
    predicciones_ponderadas = np.round(predicciones_ponderadas).astype(int)
    
    return predicciones_ponderadas

# Utilizar la función
modelos, pesos = weighted_voting_ensemble(X_trn, Y_trn, mejores_hiperparametros_GD,
                            mejores_hiperparametros_tree,mejores_hiperparametros_ADA,
                            mejores_hiperparametros_extra,mejores_hiperparametros_random,
                            mejores_hiperparametros_BG,mejores_hiperparametros_XB)
Y_pred = weighted_voting_predict(modelos, pesos, X_tst)
precision = accuracy_score(Y_tst, Y_pred)
recall = recall_score(Y_tst, Y_pred, average='weighted')
f1 = f1_score(Y_tst, Y_pred, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred)
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)
print("Precisión del modelo (Ensamblaje con voto ponderado): {:.2f}%".format(precision * 100))


Precisión:  0.8358585858585859
Exhaustividad (Recall):  0.8358585858585859
Puntuación F1:  0.8196395891767483
Exactitud:  0.8358585858585859
Precisión del modelo (Ensamblaje con voto ponderado): 83.59%


In [ ]:
resultados_df_weighted_voting = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= weighted_voting_predict(modelos, pesos, X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_weighted_voting_entrenamiento = pd.concat([resultados_df_weighted_voting, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_weighted_voting_entrenamiento["MODELO"]='VotingWeighted'
resultados_df_weighted_voting_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_weighted_voting_entrenamiento

Precisión:  94.34
Exhaustividad:  94.48
Puntuación F1:  94.04
Exactitud:  94.48
Información Mutua Normalizada (NMI): 74.96
Índice Kappa de Cohen: 89.39


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,94.34,VotingWeighted,Entrenamiento
1,Exhaustividad,94.48,VotingWeighted,Entrenamiento
2,Puntuación F1,94.04,VotingWeighted,Entrenamiento
3,Exactitud,94.48,VotingWeighted,Entrenamiento
4,Información Mutua Normalizada (NMI),74.96,VotingWeighted,Entrenamiento
5,Índice Kappa de Cohen,89.39,VotingWeighted,Entrenamiento


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_weighted_voting = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba= weighted_voting_predict(modelos, pesos, X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_weighted_voting_prueba = pd.concat([resultados_df_weighted_voting, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_weighted_voting_prueba["MODELO"]='VotingWeighted'
resultados_df_weighted_voting_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_weighted_voting_prueba

Precisión:  80.53
Exhaustividad:  83.59
Puntuación F1:  81.96
Exactitud:  83.59
Información Mutua Normalizada (NMI): 40.32
Índice Kappa de Cohen: 67.8


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.53,VotingWeighted,Prueba
1,Exhaustividad,83.59,VotingWeighted,Prueba
2,Puntuación F1,81.96,VotingWeighted,Prueba
3,Exactitud,83.59,VotingWeighted,Prueba
4,Información Mutua Normalizada (NMI),40.32,VotingWeighted,Prueba
5,Índice Kappa de Cohen,67.80,VotingWeighted,Prueba


In [ ]:
resultados_df_weighted_voting = pd.concat([resultados_df_weighted_voting_prueba,resultados_df_weighted_voting_entrenamiento], ignore_index=True)
resultados_df_weighted_voting

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.53,VotingWeighted,Prueba
1,Exhaustividad,83.59,VotingWeighted,Prueba
2,Puntuación F1,81.96,VotingWeighted,Prueba
3,Exactitud,83.59,VotingWeighted,Prueba
4,Información Mutua Normalizada (NMI),40.32,VotingWeighted,Prueba
5,Índice Kappa de Cohen,67.80,VotingWeighted,Prueba
6,Precisión,94.34,VotingWeighted,Entrenamiento
7,Exhaustividad,94.48,VotingWeighted,Entrenamiento
8,Puntuación F1,94.04,VotingWeighted,Entrenamiento
9,Exactitud,94.48,VotingWeighted,Entrenamiento


### SUPER APRENDIZ MLENS

In [40]:
Y_trn = Y_trn.astype(int)
def entrenar_modelo_super_aprendiz(X_trn, Y_trn, mejores_hiperparametros_GD,
                            mejores_hiperparametros_tree,mejores_hiperparametros_extra,
                            mejores_hiperparametros_random,mejores_hiperparametros_BG):
    X_trn_transformado = X_trn
    semilla = 7 
    kfold = StratifiedKFold(n_splits=10, random_state=semilla, shuffle=True)
    modelo1 = ExtraTreesClassifier(**mejores_hiperparametros_extra)
    modelo2 = RandomForestClassifier(**mejores_hiperparametros_random)
    model = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo3 = BaggingClassifier(**mejores_hiperparametros_BG)
    modelo4 = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo5 = GradientBoostingClassifier(**mejores_hiperparametros_GD) 
    estimadores = [('Extratrees', modelo1), ('Random Forest', modelo2), 
                ('Bagging', modelo3), ('Decision tree', modelo4),('Gradient',modelo5)]
    super_learner = SuperLearner(folds=10, random_state=semilla, verbose=2)
    super_learner.add(estimadores)
    estimador_final = ExtraTreesClassifier(n_estimators=100, max_features=None,
                                        bootstrap=False, max_depth=11, min_samples_split=4, 
                                        min_samples_leaf=1)
    super_learner.add_meta(estimador_final)
    super_learner.fit(X_trn_transformado, Y_trn)
    mejores_hiperparametros_super_learner=super_learner.get_params
    resultados = cross_val_score(super_learner, X_trn_transformado, Y_trn, cv=kfold, scoring='accuracy')
    print("Rendimiento del modelo:") 
    print("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean() * 100, "%") 
    return super_learner,mejores_hiperparametros_super_learner,estimadores
modelo_superaprendiz,mejores_hiperparametros_super_learner,estimadores = entrenar_modelo_super_aprendiz(X_trn, Y_trn, mejores_hiperparametros_GD,
                            mejores_hiperparametros_tree,mejores_hiperparametros_extra,
                            mejores_hiperparametros_random,mejores_hiperparametros_BG)


Fitting 2 layers
Processing layer-1             done | 00:01:00
Processing layer-2             done | 00:00:00
Fit complete                        | 00:01:00

Fitting 2 layers
Processing layer-1             done | 00:00:57
Processing layer-2             done | 00:00:00
Fit complete                        | 00:00:58

Predicting 2 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:00

Fitting 2 layers
Processing layer-1             done | 00:00:58
Processing layer-2             done | 00:00:00
Fit complete                        | 00:00:58

Predicting 2 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:00

Fitting 2 layers
Processing layer-1             done | 00:00:57
Processing layer-2             done | 00:00:00
Fit complete                        | 00:00:57

Predicting 2 layers
Processing layer-1 

In [ ]:
print(modelo_superaprendiz.data)

                           ft-m  ft-s  pt-m  pt-s
layer-1  bagging          21.02  0.35  1.47  0.06
layer-1  decision-tree     0.04  0.02  0.02  0.02
layer-1  extratrees        1.90  0.09  0.20  0.03
layer-1  gradient         52.37  7.19  0.01  0.00
layer-1  random-forest     1.54  0.25  0.10  0.04



In [41]:
resultados_df_superaprendiz = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_superaprendiz.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_superaprendiz_entrenamiento = pd.concat([resultados_df_superaprendiz, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_superaprendiz_entrenamiento["MODELO"]='SuperAprendiz'
resultados_df_superaprendiz_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_superaprendiz_entrenamiento


Predicting 2 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:00
Precisión:  88.59
Exhaustividad:  92.37
Puntuación F1:  90.43
Exactitud:  92.37
Información Mutua Normalizada (NMI): 69.67
Índice Kappa de Cohen: 85.13


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,88.59,SuperAprendiz,Entrenamiento
1,Exhaustividad,92.37,SuperAprendiz,Entrenamiento
2,Puntuación F1,90.43,SuperAprendiz,Entrenamiento
3,Exactitud,92.37,SuperAprendiz,Entrenamiento
4,Información Mutua Normalizada (NMI),69.67,SuperAprendiz,Entrenamiento
5,Índice Kappa de Cohen,85.13,SuperAprendiz,Entrenamiento


In [42]:
# Predecir las etiquetas para los datos de prueba
resultados_df_superaprendiz = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_superaprendiz.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_superaprendiz_prueba = pd.concat([resultados_df_superaprendiz, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_superaprendiz_prueba["MODELO"]='SuperAprendiz'
resultados_df_superaprendiz_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_superaprendiz_prueba


Predicting 2 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:00
Precisión:  79.94
Exhaustividad:  83.08
Puntuación F1:  81.48
Exactitud:  83.08
Información Mutua Normalizada (NMI): 38.89
Índice Kappa de Cohen: 66.98


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.94,SuperAprendiz,Prueba
1,Exhaustividad,83.08,SuperAprendiz,Prueba
2,Puntuación F1,81.48,SuperAprendiz,Prueba
3,Exactitud,83.08,SuperAprendiz,Prueba
4,Información Mutua Normalizada (NMI),38.89,SuperAprendiz,Prueba
5,Índice Kappa de Cohen,66.98,SuperAprendiz,Prueba


In [ ]:
hiperparametros_superaprendiz = []
for nombre, estimador in estimadores:
    hiperparametros = estimador.get_params()
    hiperparametros_estimador = {'Estimador': nombre, 'Hiperparametros': hiperparametros}
    hiperparametros_superaprendiz.append(hiperparametros_estimador)


In [ ]:
cadena_hiperparametros_superaprendiz = ', '.join([', '.join([f"{key}: {value}" for key, value in estimador.items()]) for estimador in hiperparametros_superaprendiz])

df_hiperparametros_superaprendiz = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_superaprendiz],
    'MODELO': ['SuperAprendiz'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})
resultados_df_superaprendiz = pd.concat([resultados_df_superaprendiz_prueba,resultados_df_superaprendiz_entrenamiento,df_hiperparametros_superaprendiz], ignore_index=True)
resultados_df_superaprendiz

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.7,SuperAprendiz,Prueba
1,Exhaustividad,82.83,SuperAprendiz,Prueba
2,Puntuación F1,81.23,SuperAprendiz,Prueba
3,Exactitud,82.83,SuperAprendiz,Prueba
4,Información Mutua Normalizada (NMI),38.69,SuperAprendiz,Prueba
5,Índice Kappa de Cohen,66.46,SuperAprendiz,Prueba
6,Precisión,89.28,SuperAprendiz,Entrenamiento
7,Exhaustividad,93.07,SuperAprendiz,Entrenamiento
8,Puntuación F1,91.13,SuperAprendiz,Entrenamiento
9,Exactitud,93.07,SuperAprendiz,Entrenamiento


### SUPER APRENDIZ MULTICAPA

In [ ]:
Y_trn = Y_trn.astype(int)
def entrenar_modelo_super_aprendiz_dos_capas(X_trn, Y_trn, mejores_hiperparametros_tree,
                                            mejores_hiperparametros_extra,mejores_hiperparametros_random,
                                            mejores_hiperparametros_BG):
    X_trn_transformado = X_trn
    semilla = 7 
    kfold = StratifiedKFold(n_splits=10, random_state=semilla, shuffle=True)
    modelo1 = ExtraTreesClassifier(**mejores_hiperparametros_extra)
    modelo2 = RandomForestClassifier(**mejores_hiperparametros_random)
    model = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo3 = BaggingClassifier(**mejores_hiperparametros_BG)
    modelo4 = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    estimadores = [('Extratrees', modelo1), ('Random Forest', modelo2), 
                ('Bagging', modelo3), ('Decision tree', modelo4)]
    superaprendiz_dos_capas = SuperLearner(folds=10, random_state=semilla, verbose=2)
    superaprendiz_dos_capas.add(estimadores)
    superaprendiz_dos_capas.add(estimadores)
    estimador_final = ExtraTreesClassifier(n_estimators=100, max_features=None,
                                        bootstrap=False, max_depth=11, min_samples_split=4, 
                                        min_samples_leaf=1)
    superaprendiz_dos_capas.add_meta(estimador_final)
    
    superaprendiz_dos_capas.fit(X_trn_transformado, Y_trn)
    mejores_hiperparametros_superaprendiz_dos_capas=superaprendiz_dos_capas.get_params
    resultados = cross_val_score(superaprendiz_dos_capas, X_trn_transformado, Y_trn, cv=kfold, scoring='accuracy')
    print("Rendimiento del modelo:") 
    print("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean() * 100, "%") 
    return superaprendiz_dos_capas,mejores_hiperparametros_superaprendiz_dos_capas,estimadores
modelo_superaprendiz_dos_capas,mejores_hiperparametros_superaprendiz_dos_capas,estimadores = entrenar_modelo_super_aprendiz_dos_capas(X_trn, Y_trn, mejores_hiperparametros_tree,
                                            mejores_hiperparametros_extra,mejores_hiperparametros_random,mejores_hiperparametros_BG)


Fitting 3 layers
Processing layer-1             

C:\Users\Intevo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mlens\index\base.py:83: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sizes = (n // p) * np.ones(p, dtype=np.int)
C:\Users\Intevo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mlens\index\base.py:83: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this wil

done | 00:00:20
Processing layer-2             

C:\Users\Intevo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mlens\index\base.py:83: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sizes = (n // p) * np.ones(p, dtype=np.int)
C:\Users\Intevo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mlens\index\base.py:83: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this wil

done | 00:00:15
Processing layer-3             done | 00:00:00
Fit complete                        | 00:00:36

Fitting 3 layers
Processing layer-1             

C:\Users\Intevo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mlens\index\base.py:83: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sizes = (n // p) * np.ones(p, dtype=np.int)
C:\Users\Intevo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mlens\index\base.py:83: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this wil

done | 00:00:18
Processing layer-2             

C:\Users\Intevo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mlens\index\base.py:83: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sizes = (n // p) * np.ones(p, dtype=np.int)
C:\Users\Intevo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mlens\index\base.py:83: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this wil

KeyboardInterrupt: 

In [ ]:
print(modelo_superaprendiz_dos_capas.data)

                           ft-m  ft-s  pt-m  pt-s
layer-1  bagging          16.96  0.12  1.09  0.10
layer-1  decision-tree     0.04  0.02  0.02  0.02
layer-1  extratrees        1.97  0.10  0.30  0.08
layer-1  random-forest     2.23  0.27  0.18  0.06
layer-2  bagging          14.00  0.19  0.96  0.12
layer-2  decision-tree     0.03  0.03  0.00  0.01
layer-2  extratrees        1.74  0.08  0.26  0.06
layer-2  random-forest     2.23  0.13  0.21  0.03



In [ ]:
resultados_df_superaprendiz_dos_capas = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_superaprendiz_dos_capas.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_superaprendiz_dos_capas_entrenamiento = pd.concat([resultados_df_superaprendiz_dos_capas, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_superaprendiz_dos_capas_entrenamiento["MODELO"]='SuperAprendizdoscapas'
resultados_df_superaprendiz_dos_capas_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_superaprendiz_dos_capas_entrenamiento


Predicting 3 layers


Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Processing layer-3             done | 00:00:00
Predict complete                    | 00:00:00
Precisión:  91.49
Exhaustividad:  95.34
Puntuación F1:  93.36
Exactitud:  95.34
Información Mutua Normalizada (NMI): 85.92
Índice Kappa de Cohen: 90.92


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,91.49,SuperAprendizdoscapas,Entrenamiento
1,Exhaustividad,95.34,SuperAprendizdoscapas,Entrenamiento
2,Puntuación F1,93.36,SuperAprendizdoscapas,Entrenamiento
3,Exactitud,95.34,SuperAprendizdoscapas,Entrenamiento
4,Información Mutua Normalizada (NMI),85.92,SuperAprendizdoscapas,Entrenamiento
5,Índice Kappa de Cohen,90.92,SuperAprendizdoscapas,Entrenamiento


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_superaprendiz_dos_capas = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_superaprendiz_dos_capas.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_superaprendiz_dos_capas_prueba = pd.concat([resultados_df_superaprendiz_dos_capas, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_superaprendiz_dos_capas_prueba["MODELO"]='SuperAprendizdoscapas'
resultados_df_superaprendiz_dos_capas_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_superaprendiz_dos_capas_prueba


Predicting 3 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Processing layer-3             done | 00:00:00
Predict complete                    | 00:00:00
Precisión:  79.53
Exhaustividad:  82.45
Puntuación F1:  80.83
Exactitud:  82.45
Información Mutua Normalizada (NMI): 37.1
Índice Kappa de Cohen: 65.48


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.53,SuperAprendizdoscapas,Prueba
1,Exhaustividad,82.45,SuperAprendizdoscapas,Prueba
2,Puntuación F1,80.83,SuperAprendizdoscapas,Prueba
3,Exactitud,82.45,SuperAprendizdoscapas,Prueba
4,Información Mutua Normalizada (NMI),37.10,SuperAprendizdoscapas,Prueba
5,Índice Kappa de Cohen,65.48,SuperAprendizdoscapas,Prueba


In [ ]:
hiperparametros_superaprendiz_dos_capas = [] 
for nombre, estimador in estimadores:
    hiperparametros = estimador.get_params()
    hiperparametros_estimador = {'Estimador': nombre, 'Hiperparametros': hiperparametros}
    hiperparametros_superaprendiz_dos_capas.append(hiperparametros_estimador)


In [ ]:
cadena_hiperparametros_superaprendiz_dos_capas = ', '.join([', '.join([f"{key}: {value}" for key, value in estimador.items()]) for estimador in hiperparametros_superaprendiz_dos_capas])

df_hiperparametros_superaprendiz_dos_capas = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_superaprendiz_dos_capas],
    'MODELO': ['SuperAprendizdoscapas'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})
resultados_df_superaprendiz_dos_capas = pd.concat([resultados_df_superaprendiz_dos_capas_prueba,resultados_df_superaprendiz_dos_capas_entrenamiento,df_hiperparametros_superaprendiz_dos_capas], ignore_index=True)
resultados_df_superaprendiz_dos_capas

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.53,SuperAprendizdoscapas,Prueba
1,Exhaustividad,82.45,SuperAprendizdoscapas,Prueba
2,Puntuación F1,80.83,SuperAprendizdoscapas,Prueba
3,Exactitud,82.45,SuperAprendizdoscapas,Prueba
4,Información Mutua Normalizada (NMI),37.1,SuperAprendizdoscapas,Prueba
5,Índice Kappa de Cohen,65.48,SuperAprendizdoscapas,Prueba
6,Precisión,91.49,SuperAprendizdoscapas,Entrenamiento
7,Exhaustividad,95.34,SuperAprendizdoscapas,Entrenamiento
8,Puntuación F1,93.36,SuperAprendizdoscapas,Entrenamiento
9,Exactitud,95.34,SuperAprendizdoscapas,Entrenamiento


### RED NEURONAL

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
np.random.seed(42)

torch.manual_seed(42)
torch.cuda.manual_seed(42)
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

def train_neural_network(X_train, y_train, input_size, hidden_size, num_classes, num_epochs=100, learning_rate=0.0015):
    # Convertir datos de entrenamiento a tensores de PyTorch
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    
    # Inicializar modelo y función de pérdida
    model = NeuralNetwork(input_size, hidden_size, num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Entrenamiento del modelo
    for epoch in range(num_epochs):
        # Forward pass y pérdida
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        
        # Backward pass y optimización
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (epoch+1) % 10 == 0:
            print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

    return model,num_epochs,learning_rate

def evaluate_model(model, X_test, y_test):
    # Convertir datos de prueba a tensores de PyTorch
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)

    # Obtener las predicciones del modelo
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)

    # Calcular las métricas del modelo
    accuracy = accuracy_score(y_test_tensor.numpy(), predicted.numpy())
    precision = precision_score(y_test, predicted.numpy(), average='weighted')
    recall = recall_score(y_test, predicted.numpy(), average='weighted')
    f1 = f1_score(y_test, predicted.numpy(), average='weighted')

    print('Accuracy: {:.2f}%'.format(accuracy * 100))
    print("Precisión: ", precision)
    print("Exhaustividad (Recall): ", recall)
    print("Puntuación F1: ", f1)

X_train_array = X_trn.values
X_test_array = X_tst.values
Y_trn = Y_trn 
input_size = X_trn.shape[1]
num_classes = 4
hidden_size =155

# Supongamos que X_train, y_train, X_test, y_test son tus datos de entrenamiento y prueba
modelo_red_reuronal,num_epochs,learning_rate = train_neural_network(X_train_array , Y_trn, input_size, hidden_size, num_classes)
evaluate_model(modelo_red_reuronal, X_test_array , Y_tst)


Epoch [10/100], Loss: 0.8420
Epoch [20/100], Loss: 0.6193
Epoch [30/100], Loss: 0.5383
Epoch [40/100], Loss: 0.4911
Epoch [50/100], Loss: 0.4615
Epoch [60/100], Loss: 0.4432
Epoch [70/100], Loss: 0.4294
Epoch [80/100], Loss: 0.4184
Epoch [90/100], Loss: 0.4091
Epoch [100/100], Loss: 0.4009
Accuracy: 80.43%
Precisión:  0.7772863969965955
Exhaustividad (Recall):  0.8042929292929293
Puntuación F1:  0.7879819764922228


In [ ]:
print("Tamaño de entrada:", input_size)
print("Tamaño de la capa oculta:", hidden_size)
print("Número de clases:", num_classes)
print("Número de épocas:", num_epochs)
print("Tasa de aprendizaje:", learning_rate)


Tamaño de entrada: 18
Tamaño de la capa oculta: 155
Número de clases: 4
Número de épocas: 100
Tasa de aprendizaje: 0.0015


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, normalized_mutual_info_score, cohen_kappa_score

# Predecir las etiquetas para los datos de entrenamiento
resultados_df_red_reuronal = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

# Convertir datos de entrenamiento a tensores de PyTorch
X_train_tensor = torch.tensor(X_train_array, dtype=torch.float32)

# Obtener las predicciones del modelo
outputs = modelo_red_reuronal(X_train_tensor)
_, predicted = torch.max(outputs, 1)
Y_pred_entrenamiento = predicted.numpy()

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)

print("Precisión: ", round(precision*100, 2))
print("Exhaustividad: ", round(recall*100, 2))
print("Puntuación F1: ", round(f1*100, 2))
print("Exactitud: ", round(accuracy*100, 2))
print("Información Mutua Normalizada (NMI):", round(nmi*100, 2))
print("Índice Kappa de Cohen:", round(kappa*100, 2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi = pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa = pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_red_reuronal_entrenamiento = pd.concat([resultados_df_red_reuronal, df_precision, df_recall, df_f1, df_accuracy, df_nmi, df_kappa], ignore_index=True)
resultados_df_red_reuronal_entrenamiento["MODELO"] = 'RedNeuronal'
resultados_df_red_reuronal_entrenamiento["TIPO_DE_DATOS"] = 'Entrenamiento'

# Imprimir el DataFrame con los resultados
resultados_df_red_reuronal_entrenamiento


Precisión:  81.87
Exhaustividad:  85.11
Puntuación F1:  83.33
Exactitud:  85.11
Información Mutua Normalizada (NMI): 45.42
Índice Kappa de Cohen: 70.79


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,81.87,RedNeuronal,Entrenamiento
1,Exhaustividad,85.11,RedNeuronal,Entrenamiento
2,Puntuación F1,83.33,RedNeuronal,Entrenamiento
3,Exactitud,85.11,RedNeuronal,Entrenamiento
4,Información Mutua Normalizada (NMI),45.42,RedNeuronal,Entrenamiento
5,Índice Kappa de Cohen,70.79,RedNeuronal,Entrenamiento


In [ ]:
# Predecir las etiquetas para los datos de prueba
resultados_df_red_reuronal_prueba = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

# Convertir datos de prueba a tensores de PyTorch
X_test_tensor = torch.tensor(X_test_array, dtype=torch.float32)

# Obtener las predicciones del modelo
outputs = modelo_red_reuronal(X_test_tensor)
_, predicted = torch.max(outputs, 1)
Y_pred_prueba = predicted.numpy()

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)

print("Precisión: ", round(precision*100, 2))
print("Exhaustividad: ", round(recall*100, 2))
print("Puntuación F1: ", round(f1*100, 2))
print("Exactitud: ", round(accuracy*100, 2))
print("Información Mutua Normalizada (NMI):", round(nmi*100, 2))
print("Índice Kappa de Cohen:", round(kappa*100, 2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi = pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa = pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_red_reuronal_prueba = pd.concat([resultados_df_red_reuronal_prueba, df_precision, df_recall, df_f1, df_accuracy, df_nmi, df_kappa], ignore_index=True)
resultados_df_red_reuronal_prueba["MODELO"] = 'RedNeuronal'
resultados_df_red_reuronal_prueba["TIPO_DE_DATOS"] = 'Prueba'

# Imprimir el DataFrame con los resultados
resultados_df_red_reuronal_prueba


Precisión:  77.73
Exhaustividad:  80.43
Puntuación F1:  78.8
Exactitud:  80.43
Información Mutua Normalizada (NMI): 32.56
Índice Kappa de Cohen: 61.38


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,77.73,RedNeuronal,Prueba
1,Exhaustividad,80.43,RedNeuronal,Prueba
2,Puntuación F1,78.80,RedNeuronal,Prueba
3,Exactitud,80.43,RedNeuronal,Prueba
4,Información Mutua Normalizada (NMI),32.56,RedNeuronal,Prueba
5,Índice Kappa de Cohen,61.38,RedNeuronal,Prueba


In [ ]:
# Crear un diccionario con los hiperparámetros y sus valores
hiperparametros = {
    "input_size": input_size,
    "hidden_size": hidden_size,
    "num_classes": num_classes,
    "num_epochs": num_epochs,
    "learning_rate": learning_rate
}

# Convertir el diccionario en una cadena formateada para mostrar en el dataframe
cadena_hiperparametros = ', '.join([f"{key}={value}" for key, value in hiperparametros.items()])

# Crear un nuevo dataframe con los hiperparámetros
df_hiperparametros_red_neuronal = pd.DataFrame({
    'MÉTRICA': ['Hiperparámetros'],
    'VALOR': [cadena_hiperparametros],
    'MODELO': ['RedNeuronal'],
    'TIPO_DE_DATOS': ['Entrenamiento']
})

In [ ]:
resultados_df_red_reuronal = pd.concat([resultados_df_red_reuronal_prueba,resultados_df_red_reuronal_entrenamiento,df_hiperparametros_red_neuronal], ignore_index=True)
resultados_df_red_reuronal

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,77.73,RedNeuronal,Prueba
1,Exhaustividad,80.43,RedNeuronal,Prueba
2,Puntuación F1,78.8,RedNeuronal,Prueba
3,Exactitud,80.43,RedNeuronal,Prueba
4,Información Mutua Normalizada (NMI),32.56,RedNeuronal,Prueba
5,Índice Kappa de Cohen,61.38,RedNeuronal,Prueba
6,Precisión,81.87,RedNeuronal,Entrenamiento
7,Exhaustividad,85.11,RedNeuronal,Entrenamiento
8,Puntuación F1,83.33,RedNeuronal,Entrenamiento
9,Exactitud,85.11,RedNeuronal,Entrenamiento


### Métricas

In [ ]:
Metricas_Modelos=pd.concat([resultados_df_knn,resultados_df_svc,resultados_df_tree,
                            resultados_df_gaussian,resultados_df_LDA,resultados_df_BG,
                            resultados_df_random,resultados_df_extra,resultados_df_ADA,
                            resultados_df_GD,resultados_df_XB,resultados_df_CB,
                            resultados_df_LIGHT,resultados_df_voting_hard,resultados_df_voting_soft,
                            resultados_df_stacking_lineal,resultados_df_stacking_nolineal,
                            resultados_df_weighted_voting,resultados_df_superaprendiz,
                            resultados_df_superaprendiz_dos_capas,resultados_df_red_reuronal],axis=0)
Metricas_Modelos = Metricas_Modelos.rename(columns={'MÉTRICA': 'METRICA'})
Metricas_Modelos ['METRICA'] = Metricas_Modelos ['METRICA'].apply(lambda x: unidecode(x))
#Metricas_Modelos= Metricas_Modelos[Metricas_Modelos["MODELO"].isin(modelos_seleccionados)]

In [ ]:
filtrado=Metricas_Modelos.loc[Metricas_Modelos['MODELO'].str.contains('RandomForest')]
pd.set_option('display.max_colwidth', None)
filtrado

,METRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precision,84.46,RandomForest,Prueba
1,Exhaustividad,83.84,RandomForest,Prueba
2,Puntuacion F1,82.39,RandomForest,Prueba
3,Exactitud,83.84,RandomForest,Prueba
4,Informacion Mutua Normalizada (NMI),40.5,RandomForest,Prueba
5,Indice Kappa de Cohen,68.42,RandomForest,Prueba
6,Precision,93.68,RandomForest,Entrenamiento
7,Exhaustividad,93.56,RandomForest,Entrenamiento
8,Puntuacion F1,92.93,RandomForest,Entrenamiento
9,Exactitud,93.56,RandomForest,Entrenamiento


In [ ]:
Metricas_Modelos ['MODELO'].unique()

array(['KNeighbors', 'SVC', 'DecisionTree', 'NaiveBayes', 'LDA',
       'Bagging', 'RandomForest', 'ExtraTrees', 'AdaBoost',
       'GradientBoosting', 'XGB', ' XGB', 'CatBoost', 'LGBM',
       'VotingHard', 'VotingSoft', 'StackingLineal', 'Stackingnolineal',
       'VotingWeighted', 'SuperAprendiz', 'SuperAprendizdoscapas',
       'RedNeuronal'], dtype=object)

In [ ]:
data_with_columns = Metricas_Modelos.to_dict(orient='records')

diccionario_dataframes = [
        {
            'dataTransformacion': data_with_columns,
            #'columnas': Metricas_Modelos.columns.tolist()
        }
    ]
with open("Metricas_Modelos.json", "w") as json_file:
    json.dump({"data": diccionario_dataframes}, json_file, indent=4)

    print("Los DataFrames han sido guardados en 'Metricas_Modelos.json'.")

Los DataFrames han sido guardados en 'Metricas_Modelos.json'.


In [ ]:
Metricas_Modelos.to_csv('Metricas_Modelos1.csv',sep="|",index=False,encoding='utf-8')

PermissionError: [Errno 13] Permission denied: 'Metricas_Modelos1.csv'

In [ ]:
Metricas_Modelos

,METRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precision,51.65,KNeighbors,Prueba
1,Exhaustividad,58.87,KNeighbors,Prueba
2,Puntuacion F1,53.6,KNeighbors,Prueba
3,Exactitud,58.87,KNeighbors,Prueba
4,Informacion Mutua Normalizada (NMI),0.09,KNeighbors,Prueba
...,...,...,...,...
8,Puntuacion F1,50.77,RedNeuronal,Entrenamiento
9,Exactitud,62.28,RedNeuronal,Entrenamiento
10,Informacion Mutua Normalizada (NMI),1.27,RedNeuronal,Entrenamiento
11,Indice Kappa de Cohen,5.05,RedNeuronal,Entrenamiento
